In [ ]:
# Step 1 - Setup Environment and Import Libraries
import os
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(
    filename=os.path.join(os.getcwd(), "analysis.log"),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

BASE_DIR = os.getcwd()  # Simplified for Jupyter
CACHE_DIR = os.getenv("FASTF1_CACHE_DIR", os.path.join(BASE_DIR, "fastf1_cache"))
HISTORICAL_CACHE_DIR = os.getenv("HISTORICAL_CACHE_DIR", os.path.join(BASE_DIR, "historical_cache"))
try:
    os.makedirs(CACHE_DIR, exist_ok=True)
    os.makedirs(HISTORICAL_CACHE_DIR, exist_ok=True)
except OSError as e:
    logger.error(f"Failed to create cache directory at {CACHE_DIR} or {HISTORICAL_CACHE_DIR}: {e}")
    raise RuntimeError(f"Failed to create cache directory: {e}")

fastf1.set_log_level('ERROR')
fastf1.Cache.enable_cache(CACHE_DIR)

if os.getenv("SILENT", "false").lower() != "true":
    print("Library Versions:")
    print(f"FastF1: {fastf1.__version__}")
    print(f"Pandas: {pd.__version__}")
    print(f"NumPy: {np.__version__}")
    print(f"Matplotlib: {plt.matplotlib.__version__}")
    print(f"\nCache Directory: {os.path.abspath(CACHE_DIR)}")
    print(f"Historical Cache Directory: {os.path.abspath(HISTORICAL_CACHE_DIR)}")
    print(f"Cache Directory Exists: {os.path.exists(CACHE_DIR)}")
    print(f"Historical Cache Directory Exists: {os.path.exists(HISTORICAL_CACHE_DIR)}")
    print(f"Current Date: {datetime.now().strftime('%Y-%m-%d')}")
    logger.info("Environment setup completed successfully.")

# Why: Sets up the environment with necessary libraries and caching for F1 data analysis.
# How to Read: Displays library versions and cache status to confirm setup.
# Expected Inputs: None
# Expected Outputs: Console confirmation of setup
# Data Exported: analysis.log (optional log file)

In [ ]:
# Step 2 - Determine the Target Race and Sessions
# Objective: Dynamically determine the target race, its session types, historical performance, and track metadata.

import fastf1
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display
import os
import logging

logger = logging.getLogger(__name__)

def get_target_event(manual_year=None, manual_round=None, skip_timing_validation=False):
    current_date = datetime.utcnow()
    year = manual_year if manual_year is not None else current_date.year
    schedule = fastf1.get_event_schedule(year)
    print(f"Total Events in {year} Schedule: {len(schedule)}")
    logger.info(f"Fetched {year} schedule with {len(schedule)} events.")

    if manual_round is not None and manual_year is not None:
        target_event = schedule[schedule['RoundNumber'] == manual_round].iloc[0]
    else:
        future_events = schedule[schedule['EventDate'] >= current_date - timedelta(days=7)]
        target_event = future_events.iloc[0] if not future_events.empty else schedule.iloc[-1]

    round_number = target_event['RoundNumber']
    event_name = target_event['EventName']
    event_date = target_event['EventDate'].strftime('%Y-%m-%d')
    is_sprint = 'sprint' in target_event['EventFormat'].lower()
    location = target_event['Location']

    session_types = []
    session_dates = {}
    for session in ['Session1', 'Session2', 'Session3']:
        session_name = target_event[session]
        if session_name in ['Practice 1', 'Practice 2', 'Practice 3', 'Sprint Qualifying', 'Sprint']:
            session_types.append(session_name)
            session_dates[session_name] = target_event[f"{session}DateUtc"]
    if not session_types or session_types[-1] not in ['Practice 3', 'Sprint']:
        logger.error(f"No valid final session found for {event_name}")
        raise ValueError(f"No valid final session found for {event_name}")

    if not skip_timing_validation:
        final_session_name = session_types[-1]
        final_session_end = session_dates[final_session_name]
        qualifying_date_key = 'Session4DateUtc' if not is_sprint else 'Session5DateUtc'
        qualifying_start = target_event[qualifying_date_key] if qualifying_date_key in target_event else None
        if final_session_end and current_date < final_session_end:
            logger.warning(f"Current time {current_date} is before {final_session_name} end {final_session_end}.")
            raise ValueError(f"Analysis must run after {final_session_name}.")
        if qualifying_start and current_date > qualifying_start:
            logger.warning(f"Current time {current_date} is after Qualifying start {qualifying_start}.")
            raise ValueError("Analysis must run before Qualifying starts.")

    historical_winners = []
    historical_poles = []
    try:
        for past_year in range(year-3, year):
            past_schedule = fastf1.get_event_schedule(past_year)
            past_event = past_schedule[past_schedule['Location'] == location]
            if not past_event.empty:
                past_event = past_event.iloc[0]
                past_race = fastf1.get_session(past_year, past_event['RoundNumber'], 'R')
                past_race.load()
                past_qualifying = fastf1.get_session(past_year, past_event['RoundNumber'], 'Q')
                past_qualifying.load()
                winner = past_race.results.iloc[0]['Abbreviation']
                pole_sitter = past_qualifying.results.iloc[0]['Abbreviation']
                weight = 3 - (year - past_year)
                historical_winners.append((past_year, winner, weight))
                historical_poles.append((past_year, pole_sitter, weight))
    except Exception as e:
        logger.warning(f"Failed to fetch historical data for {location}: {e}")
        historical_winners = [("N/A", "Unknown", 1)]
        historical_poles = [("N/A", "Unknown", 1)]

    # Dynamic track metadata (placeholders; computed in a later cell)
    track_length = None  # From telemetry (e.g., FP1)
    avg_speed = None     # From lap times (e.g., FP1)
    downforce_level = None  # To be inferred from telemetry (e.g., corner count vs. straight speed)

    event_details = pd.DataFrame({
        'Round': [round_number],
        'Event Name': [event_name],
        'Event Date': [event_date],
        'Location': [location],
        'Sprint Weekend': [is_sprint],
        'Sessions': [', '.join(session_types)],
        'Track Length (km)': [track_length if track_length else 'TBD'],
        'Avg Speed (km/h)': [avg_speed if avg_speed else 'TBD'],
        'Downforce Level': [downforce_level if downforce_level else 'TBD']
    })

    historical_winners_df = pd.DataFrame(historical_winners, columns=['Year', 'Winner', 'Weight'])
    historical_poles_df = pd.DataFrame(historical_poles, columns=['Year', 'Pole Sitter', 'Weight'])

    print("\nTarget Event Details (Track data TBD until session loaded):")
    display(event_details)
    print("\nHistorical Winners at this Track:")
    display(historical_winners_df)
    print("\nHistorical Pole Sitters at this Track:")
    display(historical_poles_df)

    if os.getenv("SILENT", "false").lower() != "true":
        print("\nFull Event Row from Schedule:")
        print(target_event)

    result = {
        'year': year,
        'round': round_number,
        'session_types': session_types,
        'event_name': event_name,
        'event_date': event_date,
        'location': location,
        'historical_winners': historical_winners,
        'historical_poles': historical_poles,
        'track_length': track_length,
        'avg_speed': avg_speed,
        'downforce_level': downforce_level
    }
    print("Debug - Step 2 result:", result)
    logger.info(f"Target event selected: {event_name} (Round {round_number}) with sessions {session_types}")
    return result

target_info = get_target_event(manual_year=2024, manual_round=3, skip_timing_validation=True)

# Why: Identifies the target race and sessions for pre-qualifying analysis with historical and track context.
# How to Read: Displays the selected race, its sessions, historical performance with weights, and track metadata (TBD until computed) to inform predictions.
# Expected Inputs: Optional manual_year, manual_round, skip_timing_validation
# Expected Outputs: Event schedule length, event details table (track data TBD), historical data with weights, full event row, structured result
# Data Exported: Dictionary with race and session details, track data placeholders

In [ ]:
# Step 3 - Download Practice Session Data
# Objective: Retrieve data for all practice sessions, extract key indicators, compute track metadata, save as CSV, and preview for verification.

import fastf1
import pandas as pd
import numpy as np
import os
from IPython.display import display
import logging

logger = logging.getLogger(__name__)

def download_practice_data(target_info):
    year = target_info['year']
    round_number = target_info['round']
    session_labels = target_info['session_types']
    event_name = target_info['event_name']

    # Directory to save raw data (structured as raw_data/{year}/R{round_number}/)
    BASE_DIR = os.getcwd()
    data_dir = os.path.join(BASE_DIR, "raw_data", str(year), f"R{round_number}")
    os.makedirs(data_dir, exist_ok=True)  # Ensure directory exists

    # Debug: Verify event schedule
    schedule = fastf1.get_event_schedule(year)
    print(f"Debug - {year} Schedule Length: {len(schedule)}")
    event = schedule[schedule['RoundNumber'] == round_number]
    if event.empty:
        raise ValueError(f"No event found for Round {round_number} in {year}. Available rounds: {schedule['RoundNumber'].tolist()}")
    print(f"Debug - Selected Event: {event['EventName'].iloc[0]} (Round {round_number})")

    # Dictionary to store session data
    session_data = {}

    # Compute track metadata using FP1
    try:
        fp1_session = fastf1.get_session(year, event_name, 'FP1')
        fp1_session.load(telemetry=True)
        # Use accurate, full laps for track length
        valid_laps = fp1_session.laps[fp1_session.laps['IsAccurate'] & fp1_session.laps['LapTime'].notna()]
        top_laps = valid_laps.nlargest(5, 'LapTime')  # Top 5 accurate laps
        lap_lengths = []
        for _, lap in top_laps.iterrows():
            telemetry = lap.get_telemetry()
            lap_length = (telemetry['Distance'].max() - telemetry['Distance'].min()) / 1000  # km
            lap_lengths.append(lap_length)
        track_length = np.mean(lap_lengths)  # Average for consistency
        target_info['track_length'] = track_length

        # Average speed: based on fastest lap and computed track length
        fastest_lap = fp1_session.laps.pick_fastest()
        lap_time_seconds = fastest_lap['LapTime'].total_seconds()
        avg_speed = (track_length / (lap_time_seconds / 3600))  # km/h
        target_info['avg_speed'] = avg_speed

        # Downforce level (heuristic)
        telemetry = fastest_lap.get_telemetry()
        speeds = telemetry['Speed']
        top_speed = speeds.max()
        corner_speeds = speeds[telemetry['Throttle'] < 90]  # Proxy for corners
        avg_corner_speed = corner_speeds.mean() if not corner_speeds.empty else top_speed / 2
        corner_to_straight_ratio = avg_corner_speed / top_speed
        circuit_info = fp1_session.get_circuit_info()
        corner_count = len(circuit_info.corners)
        if corner_count > 15 and corner_to_straight_ratio > 0.7:
            downforce_level = 'High'
        elif corner_count > 10 and corner_to_straight_ratio > 0.5:
            downforce_level = 'Medium-High'
        elif corner_count > 5 and corner_to_straight_ratio > 0.3:
            downforce_level = 'Medium'
        else:
            downforce_level = 'Low'
        target_info['downforce_level'] = downforce_level

        print(f"\nComputed Track Metadata (from FP1):")
        print(f"Track Length: {track_length:.3f} km")
        print(f"Average Speed: {avg_speed:.1f} km/h")
        print(f"Downforce Level: {downforce_level}")
    except Exception as e:
        logger.error(f"Failed to compute track metadata: {e}")
        print(f"Warning: Track metadata computation failed: {e}")
        target_info['track_length'] = target_info.get('track_length', 'TBD')
        target_info['avg_speed'] = target_info.get('avg_speed', 'TBD')
        target_info['downforce_level'] = target_info.get('downforce_level', 'TBD')

    # Download and process each session
    for session_num, session_label in enumerate(session_labels, start=1):
        print(f"\nDownloading Session {session_num} ({session_label}) data...")
        session = fastf1.get_session(year, event_name, session_label)
        session.load(telemetry=True)
        logger.info(f"Loaded data for {session_label} with {len(session.laps)} laps.")

        # Extract lap data
        laps = session.laps
        session_data[session_label] = {'laps': laps}

        # Save lap data
        file_path = os.path.join(data_dir, f"{year}_R{round_number}_{session_label.replace(' ', '_')}_laps.csv")
        laps.to_csv(file_path, index=False)
        print(f"Saved {session_label} lap data to {file_path}")

        # Extract weather data
        weather = session.weather_data
        session_data[session_label]['weather'] = weather
        weather_file_path = os.path.join(data_dir, f"{year}_R{round_number}_{session_label.replace(' ', '_')}_weather.csv")
        weather.to_csv(weather_file_path, index=False)
        print(f"Saved {session_label} weather data to {weather_file_path}")

        # Compute indicators
        fastest_laps = laps.groupby('Driver')['LapTime'].min().reset_index()
        avg_lap_times = laps.groupby('Driver')['LapTime'].mean().reset_index().rename(columns={'LapTime': 'AvgLapTime'})
        tyre_degradation = laps[laps['Compound'].notna() & laps['TyreLife'].notna()]
        tyre_degradation = tyre_degradation.groupby(['Driver', 'Stint']).agg({
            'LapTime': lambda x: x.mean().total_seconds() if not x.empty else np.nan,
            'TyreLife': 'mean'
        }).reset_index()
        # Refined degradation rate: tighter clip and fill NaN
        tyre_degradation['DegradationRate'] = (
            tyre_degradation.groupby('Driver')
            .apply(lambda x: (x['LapTime'].diff() / x['TyreLife'].diff()).replace([np.inf, -np.inf], 0).clip(lower=-0.1, upper=0.1), include_groups=False)
            .reset_index(drop=True)
        )
        tyre_degradation = tyre_degradation.groupby('Driver')['DegradationRate'].mean().fillna(0).reset_index()
        indicators = fastest_laps.merge(avg_lap_times, on='Driver').merge(tyre_degradation, on='Driver', how='left')
        session_data[session_label]['indicators'] = indicators
        indicators_file_path = os.path.join(data_dir, f"{year}_R{round_number}_{session_label.replace(' ', '_')}_indicators.csv")
        indicators.to_csv(indicators_file_path, index=False)
        print(f"Saved {session_label} indicators to {indicators_file_path}")

        # Preview data
        if os.getenv("SILENT", "false").lower() != "true":
            print(f"\n{session_label} Lap Data Preview (first 5 rows):")
            display(laps.head())
            print(f"\n{session_label} Weather Data Preview (first 5 rows):")
            display(weather.head())
            print(f"\n{session_label} Indicators (Fastest Lap, Avg Lap Time, Tyre Degradation):")
            display(indicators)

    # Summary
    if os.getenv("SILENT", "false").lower() != "true":
        print("\nDownload Summary:")
        for session_label, data in session_data.items():
            print(f"{session_label}: {len(data['laps'])} laps, Columns: {list(data['laps'].columns)}")
            print(f"  Weather: {len(data['weather'])} records")
            print(f"  Indicators: {len(data['indicators'])} drivers")

    return session_data

# Example usage
session_data = download_practice_data(target_info)

# Expected Inputs: target_info dict from Step 2 (year, round, session_types, event_name)
# Expected Outputs: Track metadata, download confirmations, file paths, data previews, summary (unless SILENT=true)
# Data Exported: CSV files for each session (laps, weather, indicators) in 'raw_data', updated target_info

In [ ]:
# Step 4 - Clean and Aggregate Practice Data
# Objective: Clean lap data, aggregate driver stats across sessions, and prepare for predictions.

import pandas as pd
import os
from IPython.display import display
import logging

logger = logging.getLogger(__name__)

def clean_and_aggregate_data(target_info, session_data):
    year = target_info['year']
    round_number = target_info['round']
    session_types = target_info['session_types']
    BASE_DIR = os.getcwd()  # Jupyter-friendly
    DATA_DIR = os.path.join(BASE_DIR, "raw_data", str(year), f"R{round_number}")
    CLEANED_DIR = os.path.join(DATA_DIR, "cleaned_data")
    os.makedirs(CLEANED_DIR, exist_ok=True)

    if not session_data:
        logger.error("No session data provided from Step 3.")
        return {}, pd.DataFrame()

    cleaned_data = {}
    driver_summary = []
    total_raw_laps = 0
    total_cleaned_laps = 0

    for session_type, data in session_data.items():
        logger.info(f"Cleaning {session_type} data...")
        laps = data['laps']  # Use laps from session_data
        if laps.empty:
            logger.warning(f"{session_type} data is empty.")
            continue
        total_raw_laps += len(laps)
        logger.info(f"Raw Laps in {session_type}: {len(laps)}")

        # Clean laps: remove deleted, use accurate laps, drop incomplete
        clean_laps = laps[(laps['Deleted'] == False) & (laps['IsAccurate'] == True)].copy()
        logger.info(f"After Deleting Removed/Keeping Accurate: {len(clean_laps)}")
        clean_laps = clean_laps.dropna(subset=['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time'], how='all')
        logger.info(f"After Dropping Fully NaN Laps: {len(clean_laps)}")

        # Convert LapTime to seconds
        clean_laps['LapTimeSeconds'] = pd.to_timedelta(clean_laps['LapTime'], errors='coerce').dt.total_seconds()
        clean_laps = clean_laps[clean_laps['LapTimeSeconds'].notna() & (clean_laps['LapTimeSeconds'] > 0)]
        total_cleaned_laps += len(clean_laps)

        # Filter outliers and pit laps for average lap time
        clean_for_avg = clean_laps[
            (clean_laps['PitInTime'].isna()) &
            (clean_laps['PitOutTime'].isna())
        ]
        driver_medians = clean_for_avg.groupby('Driver')['LapTimeSeconds'].median()
        clean_for_avg = clean_for_avg.merge(driver_medians, on='Driver', suffixes=('', '_Median'))
        clean_for_avg = clean_for_avg[clean_for_avg['LapTimeSeconds'] <= clean_for_avg['LapTimeSeconds_Median'] * 1.3].drop(columns=['LapTimeSeconds_Median'])
        logger.info(f"Laps for AvgLapTime (After Pit/Outlier Filter): {len(clean_for_avg)}")

        # Store cleaned data
        cleaned_data[session_type] = clean_laps

        # Driver stats per session
        driver_stats = clean_for_avg.groupby('Driver').agg({
            'LapTimeSeconds': ['min', 'mean', 'std', 'count']
        }).reset_index()
        driver_stats.columns = ['Driver', 'FastestLap', 'AvgLapTime', 'LapTimeStd', 'LapCount']
        driver_stats['Session'] = session_type
        driver_summary.append(driver_stats)

        # Save cleaned laps
        cleaned_file = os.path.join(CLEANED_DIR, f"{year}_R{round_number}_{session_type.replace(' ', '_')}_cleaned_laps.csv")
        clean_laps.to_csv(cleaned_file, index=False)
        logger.info(f"Saved cleaned laps to {cleaned_file}")

    if not driver_summary:
        logger.error("No driver data aggregated.")
        overall_summary = pd.DataFrame()
    else:
        all_driver_summary = pd.concat(driver_summary, ignore_index=True)
        overall_summary = all_driver_summary.groupby('Driver').agg({
            'FastestLap': 'min',
            'AvgLapTime': 'mean',
            'LapTimeStd': 'mean',
            'LapCount': 'sum'
        }).reset_index()
        overall_summary = overall_summary.sort_values('FastestLap')
        if os.getenv("SILENT", "false").lower() != "true":
            logger.info(f"Total Raw Laps: {total_raw_laps}")
            logger.info(f"Total Cleaned Laps: {total_cleaned_laps}")
            logger.info("Driver Summary Generated (Top 5):")
            print(overall_summary.head().to_string(index=False))
            logger.info("Full Overall Driver Summary:")
            display(overall_summary)

    output_file = os.path.join(CLEANED_DIR, f"{year}_R{round_number}_practice_summary.csv")
    overall_summary.to_csv(output_file, index=False)
    logger.info(f"Saved aggregated practice summary to {output_file}")

    logger.info(f"Cleaned data keys: {list(cleaned_data.keys())}")
    return cleaned_data, overall_summary

# Example usage
cleaned_data, driver_summary = clean_and_aggregate_data(target_info, session_data)

# Why: Cleans raw lap data and aggregates driver performance stats for prediction.
# How to Read: Shows cleaned lap counts, driver stats per session, and an overall summary sorted by fastest lap.
# Expected Inputs: target_info (year, round, session_types), session_data from Step 3
# Expected Outputs: Cleaned lap CSVs, practice summary CSV, previews of top 5 and full summary
# Data Exported: Cleaned laps per session, overall driver summary

In [ ]:
# Step 5: Unified Driver Performance Analysis
# Objective: Compute fastest lap times, best sector times, base pace, variance, degradation slopes, and theoretical fastest lap, with visualizations and a dynamic conclusion.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def compute_driver_performance(cleaned_data, year, round_number, data_dir="raw_data"):
    """
    Compute fastest lap times, best sector times, base pace, variance, degradation slopes, and theoretical fastest lap.

    Args:
        cleaned_data (dict): Cleaned lap DataFrames from Step 4
        year (int): Race year (e.g., 2024)
        round_number (int): Round number (e.g., 3)
        data_dir (str): Directory for output files (default: "raw_data")

    Returns:
        pd.DataFrame: Driver performance metrics
    """
    # Update data_dir to match the format raw_data/{year}/R{round_number}/
    data_dir = os.path.join(data_dir, str(year), f"R{round_number}")

    # Combine all practice sessions
    all_laps = pd.concat([cleaned_data[session] for session in cleaned_data.keys()], ignore_index=True)

    # Compute fastest lap time per driver
    fastest_laps = all_laps.groupby('Driver').agg({
        'LapTimeSeconds': 'min',
        'Team': 'first'
    }).reset_index()
    fastest_laps = fastest_laps.rename(columns={'LapTimeSeconds': 'FastestLapTime'})

    # Compute best sector times per driver
    all_laps['Sector1Time'] = pd.to_timedelta(all_laps['Sector1Time'], errors='coerce').dt.total_seconds()
    all_laps['Sector2Time'] = pd.to_timedelta(all_laps['Sector2Time'], errors='coerce').dt.total_seconds()
    all_laps['Sector3Time'] = pd.to_timedelta(all_laps['Sector3Time'], errors='coerce').dt.total_seconds()
    sector_performance = all_laps.groupby('Driver').agg({
        'Sector1Time': 'min',
        'Sector2Time': 'min',
        'Sector3Time': 'min'
    }).reset_index()

    # Compute theoretical fastest lap
    sector_performance['TheoreticalLapTime'] = sector_performance['Sector1Time'] + sector_performance['Sector2Time'] + sector_performance['Sector3Time']

    # Compute base pace and variance from long-run laps
    long_run_laps = all_laps[(all_laps['TyreLife'] > 10) & (all_laps['IsAccurate'])]
    if not long_run_laps.empty:
        # Ensure sufficient long-run laps per driver
        long_run_counts = long_run_laps.groupby('Driver').size()
        insufficient_drivers = long_run_counts[long_run_counts < 3].index.tolist()
        if insufficient_drivers:
            print(f"Warning: Insufficient long-run laps (<3) for drivers: {', '.join(insufficient_drivers)}. Using fallback for these drivers.")

        # Filter outliers using IQR
        Q1 = long_run_laps['LapTimeSeconds'].quantile(0.25)
        Q3 = long_run_laps['LapTimeSeconds'].quantile(0.75)
        IQR = Q3 - Q1
        long_run_laps = long_run_laps[
            (long_run_laps['LapTimeSeconds'] >= Q1 - 1.5 * IQR) &
            (long_run_laps['LapTimeSeconds'] <= Q3 + 1.5 * IQR)
        ]
        base_pace = long_run_laps.groupby('Driver')['LapTimeSeconds'].mean().reset_index()
        base_pace.columns = ['Driver', 'BasePace']
        pace_variance = long_run_laps.groupby('Driver')['LapTimeSeconds'].std().reset_index()
        pace_variance.columns = ['Driver', 'LapTimeVar']

        # Compute degradation slope
        deg_slope = long_run_laps.groupby(['Driver', 'Compound'], as_index=False).apply(
            lambda x: pd.Series({'DegradationSlope': max(7.0, np.polyfit(x['TyreLife'], x['LapTimeSeconds'], 1)[0] * 100) if len(x) > 2 else 7.0}),
            include_groups=False
        ).reset_index(drop=True)
        # Use the primary compound's degradation slope
        deg_slope = deg_slope.groupby('Driver').agg({'DegradationSlope': 'mean'}).reset_index()
    else:
        base_pace = pd.DataFrame({'Driver': fastest_laps['Driver'], 'BasePace': fastest_laps['FastestLapTime'] + 5})  # Fallback
        pace_variance = pd.DataFrame({'Driver': fastest_laps['Driver'], 'LapTimeVar': 0.01 * base_pace['BasePace']})
        deg_slope = pd.DataFrame({'Driver': fastest_laps['Driver'], 'DegradationSlope': 7.0})

    # Merge all metrics with outer joins to retain all drivers
    driver_performance = fastest_laps.merge(sector_performance, on='Driver', how='left')
    driver_performance = driver_performance.merge(base_pace, on='Driver', how='outer')
    driver_performance = driver_performance.merge(pace_variance, on='Driver', how='outer')
    driver_performance = driver_performance.merge(deg_slope, on='Driver', how='outer')

    # Apply fallbacks for drivers with insufficient long-run data
    mask = driver_performance['BasePace'].isna()
    driver_performance.loc[mask, 'BasePace'] = driver_performance.loc[mask, 'FastestLapTime'] + 5
    driver_performance.loc[mask, 'LapTimeVar'] = 0.01 * driver_performance.loc[mask, 'BasePace']
    driver_performance.loc[mask, 'DegradationSlope'] = 7.0

    # Ensure realistic minimums
    driver_performance['DegradationSlope'] = driver_performance['DegradationSlope'].clip(lower=7.0)
    driver_performance['LapTimeVar'] = driver_performance['LapTimeVar'].clip(lower=0.01 * driver_performance['BasePace'])

    # Sort by TheoreticalLapTime
    driver_performance = driver_performance.sort_values('TheoreticalLapTime')

    # Display table with sector times and theoretical lap time
    print("\nDriver Performance (sorted by TheoreticalLapTime):")
    display(driver_performance[['Driver', 'FastestLapTime', 'TheoreticalLapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'BasePace', 'DegradationSlope', 'Team']])

    # Visualization 1: Bar chart comparing FastestLapTime and TheoreticalLapTime
    plt.figure(figsize=(12, 6))
    bar_width = 0.35
    index = range(len(driver_performance))
    min_time = driver_performance['TheoreticalLapTime'].min() - 1
    max_time = driver_performance['TheoreticalLapTime'].max() + 1
    plt.bar([i - bar_width/2 for i in index], driver_performance['FastestLapTime'], bar_width, label='Fastest Lap Time', color='blue')
    plt.bar([i + bar_width/2 for i in index], driver_performance['TheoreticalLapTime'], bar_width, label='Theoretical Lap Time', color='orange')
    plt.xlabel('Driver')
    plt.ylabel('Lap Time (seconds)')
    plt.title(f'Fastest vs. Theoretical Lap Times ({year} Round {round_number})')
    plt.xticks(index, driver_performance['Driver'], rotation=45, ha='right')
    plt.ylim(min_time, max_time)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Visualization 2: Separate bar charts for each sector time
    sectors = ['Sector1Time', 'Sector2Time', 'Sector3Time']
    sector_titles = ['Sector 1 Time', 'Sector 2 Time', 'Sector 3 Time']
    for sector, title in zip(sectors, sector_titles):
        plt.figure(figsize=(12, 6))
        sorted_sector = driver_performance.sort_values(sector)
        min_sector = sorted_sector[sector].min() - 0.5
        max_sector = sorted_sector[sector].max() + 0.5
        plt.bar(sorted_sector['Driver'], sorted_sector[sector], color='blue')
        plt.xlabel('Driver')
        plt.ylabel('Sector Time (seconds)')
        plt.title(f'Best {title} per Driver ({year} Round {round_number})')
        plt.xticks(rotation=45, ha='right')
        plt.ylim(min_sector, max_sector)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    # Visualization 3: Bar chart for BasePace
    plt.figure(figsize=(12, 6))
    sorted_base_pace = driver_performance.sort_values('BasePace')
    min_pace = sorted_base_pace['BasePace'].min() - 1
    max_pace = sorted_base_pace['BasePace'].max() + 1
    plt.bar(sorted_base_pace['Driver'], sorted_base_pace['BasePace'], color='green')
    plt.xlabel('Driver')
    plt.ylabel('Base Pace (seconds)')
    plt.title(f'Base Pace per Driver ({year} Round {round_number})')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(min_pace, max_pace)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Visualization 4: Scatter plot of DegradationSlope vs. BasePace
    plt.figure(figsize=(12, 6))
    min_slope = driver_performance['DegradationSlope'].min() - 2
    max_slope = driver_performance['DegradationSlope'].max() + 2
    min_pace_scatter = driver_performance['BasePace'].min() - 1
    max_pace_scatter = driver_performance['BasePace'].max() + 1
    plt.scatter(driver_performance['BasePace'], driver_performance['DegradationSlope'], c='red', label='Drivers')
    for i, driver in enumerate(driver_performance['Driver']):
        plt.text(driver_performance['BasePace'].iloc[i], driver_performance['DegradationSlope'].iloc[i], driver, fontsize=7, ha='right')
    plt.xlabel('Base Pace (seconds)')
    plt.ylabel('Degradation Slope (% per lap)')
    plt.title(f'Degradation Slope vs. Base Pace ({year} Round {round_number})')
    plt.ylim(min_slope, max_slope)
    plt.xlim(min_pace_scatter, max_pace_scatter)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Visualization 5: Boxplot for BasePace per Driver
    plt.figure(figsize=(12, 6))
    base_pace_boxes = [long_run_laps[long_run_laps['Driver'] == driver]['LapTimeSeconds'].dropna() for driver in driver_performance['Driver']]
    plt.boxplot(base_pace_boxes, tick_labels=driver_performance['Driver'], patch_artist=True)
    plt.ylabel('Base Pace (seconds)')
    plt.title(f'Boxplot of Base Pace per Driver ({year} Round {round_number})')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Visualization 6: Bar chart for DegradationSlope per Driver (using calculated values)
    print("\nDebug: DegradationSlope values for plot:", driver_performance['DegradationSlope'].tolist())
    plt.figure(figsize=(12, 6))
    sorted_deg_slope = driver_performance.sort_values('DegradationSlope', ascending=False)
    plt.bar(sorted_deg_slope['Driver'], sorted_deg_slope['DegradationSlope'], color='purple')
    plt.xlabel('Driver')
    plt.ylabel('Degradation Slope (% per lap)')
    plt.title(f'Degradation Slope per Driver ({year} Round {round_number})')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(6, 20)  # Set y-axis to match DegradationSlope range
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Dynamic Conclusion and Guide
    print("\n=== How to Read the Output ===")
    print("- **FastestLapTime**: The fastest actual lap time achieved by each driver across practice sessions, indicating peak performance potential for qualifying.")
    print("- **TheoreticalLapTime**: The sum of the best sector times (Sector1Time + Sector2Time + Sector3Time), representing the theoretical fastest lap if the driver perfectly combines their best sectors.")
    print("- **Sector1Time, Sector2Time, Sector3Time**: The best times for each sector, showing strengths in different parts of the track.")
    print("- **BasePace**: The average lap time on long runs (tyre life > 10 laps), reflecting race pace consistency.")
    print("- **DegradationSlope**: The rate of lap time increase per lap due to tyre degradation, higher values indicate faster tyre wear.")
    print("- **Boxplot of BasePace per Driver**: Shows the distribution of long-run lap times for each driver, with median, quartiles, and outliers indicating race pace consistency.")
    print("- **Bar Chart of DegradationSlope per Driver**: Displays the individual degradation slope values per driver, sorted from highest to lowest, highlighting tyre wear rates.")

    print("\n=== Conclusion ===")
    print("This analysis provides insights into driver performance across practice sessions:")
    # Highlight fastest drivers
    fastest_driver = driver_performance.loc[driver_performance['FastestLapTime'].idxmin(), 'Driver']
    fastest_time = driver_performance['FastestLapTime'].min()
    print(f"- **Fastest Lap**: {fastest_driver} achieved the fastest lap time of {fastest_time:.2f} seconds, indicating strong qualifying potential.")

    # Highlight theoretical fastest lap
    theoretical_fastest_driver = driver_performance.loc[driver_performance['TheoreticalLapTime'].idxmin(), 'Driver']
    theoretical_fastest_time = driver_performance['TheoreticalLapTime'].min()
    print(f"- **Theoretical Fastest Lap**: {theoretical_fastest_driver} could theoretically achieve {theoretical_fastest_time:.2f} seconds by combining their best sector times.")

    # Highlight largest gap between fastest and theoretical lap
    driver_performance['LapTimeGap'] = driver_performance['FastestLapTime'] - driver_performance['TheoreticalLapTime']
    largest_gap_driver = driver_performance.loc[driver_performance['LapTimeGap'].idxmax(), 'Driver']
    largest_gap = driver_performance['LapTimeGap'].max()
    print(f"- **Largest Gap**: {largest_gap_driver} has the largest gap between their fastest lap and theoretical lap ({largest_gap:.2f} seconds), indicating inconsistency across sectors.")

    # Highlight base pace anomalies dynamically
    base_pace_median = driver_performance['BasePace'].median()
    base_pace_q1 = driver_performance['BasePace'].quantile(0.25)
    base_pace_q3 = driver_performance['BasePace'].quantile(0.75)
    base_pace_outliers_low = driver_performance[driver_performance['BasePace'] < base_pace_q1 - 1.5 * (base_pace_q3 - base_pace_q1)]['Driver'].tolist()
    if base_pace_outliers_low:
        for driver in base_pace_outliers_low:
            driver_base_pace = driver_performance[driver_performance['Driver'] == driver]['BasePace'].iloc[0]
            print(f"- **Base Pace Anomaly**: {driver}’s base pace ({driver_base_pace:.2f} seconds) is significantly lower than the median ({base_pace_median:.2f} seconds), possibly due to a setup optimized for short runs or insufficient long-run data. Verify with additional laps.")

    # Highlight race pace insights from boxplot
    base_pace_outliers = driver_performance[(driver_performance['BasePace'] < base_pace_q1 - 1.5 * (base_pace_q3 - base_pace_q1)) |
                                           (driver_performance['BasePace'] > base_pace_q3 + 1.5 * (base_pace_q3 - base_pace_q1))]['Driver'].tolist()
    print(f"- **Race Pace Insight**: The median base pace is {base_pace_median:.2f} seconds, with outliers including {', '.join(base_pace_outliers) if base_pace_outliers else 'none'}, indicating potential inconsistency or unique setups.")

    # Highlight tyre degradation insights dynamically
    deg_slope_median = driver_performance['DegradationSlope'].median()
    deg_slope_q1 = driver_performance['DegradationSlope'].quantile(0.25)
    deg_slope_q3 = driver_performance['DegradationSlope'].quantile(0.75)
    deg_slope_outliers = driver_performance[(driver_performance['DegradationSlope'] < deg_slope_q1 - 1.5 * (deg_slope_q3 - deg_slope_q1)) |
                                           (driver_performance['DegradationSlope'] > deg_slope_q3 + 1.5 * (deg_slope_q3 - deg_slope_q1))]['Driver'].tolist()
    print(f"- **Tyre Degradation Insight**: The median degradation slope is {deg_slope_median:.2f}% per lap, with outliers including {', '.join(deg_slope_outliers) if deg_slope_outliers else 'none'}, suggesting potential tyre wear issues.")

    print("\n**Next Steps**: Compare these metrics with telemetry data (Step 6) to assess car setup, sandbagging analysis (Step 7) for hidden pace, and track characteristics (Step 9) to understand track-specific advantages.")

    # Save to CSV
    output_file = os.path.join(data_dir, f"{year}_R{round_number}_driver_performance.csv")
    driver_performance.to_csv(output_file, index=False)
    print(f"Saved driver performance to {output_file}")

    return driver_performance

# Example usage with dynamic target_info
driver_performance_df = compute_driver_performance(cleaned_data, year=target_info['year'], round_number=target_info['round'])

# Why: Analyzes driver performance metrics (fastest lap, sectors, base pace, degradation) for prediction.
# How to Read: Displays a table of metrics, visualizations (bar charts, scatter, boxplot), and a dynamic conclusion.
# Expected Inputs: cleaned_data from Step 4, year and round_number from target_info
# Expected Outputs: Table, 6 visualizations, conclusion, CSV file
# Data Exported: driver_performance.csv in raw_data/{year}/R{round_number}/ directory

In [ ]:
# Step 6 - Assess Telemetry Metrics
# Objective: Analyze telemetry (top speed, throttle time, braking) to infer car setup and driver style, visualizing as bar charts.

import pandas as pd
import matplotlib.pyplot as plt
import os
import fastf1
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def analyze_telemetry_metrics(config):
    """
    Analyze telemetry metrics (top speed, throttle time, braking) to infer car setup and driver style.

    Args:
        config (dict): Configuration dictionary containing:
            - 'year': The race year (e.g., 2024)
            - 'round': The round number (e.g., 3)
            - 'session_types': List of session labels (default: ['Practice 1', 'Practice 2', 'Practice 3'])
            - 'data_dir': Directory for raw data (default: "raw_data")
            - 'event_name': Event name (optional, for context)

    Returns:
        pd.DataFrame: DataFrame with averaged telemetry metrics
    """
    # Extract configuration
    year = config.get('year')
    round_number = config.get('round')  # Matches target_info key
    session_labels = config.get('session_types', ['Practice 1', 'Practice 2', 'Practice 3'])
    data_dir = os.path.join("raw_data", str(year), f"R{round_number}")  # Updated to match project structure
    event_name = config.get('event_name', "Unknown Event")

    # Set base directory dynamically
    BASE_DIR = os.getcwd()  # Jupyter-friendly, avoids __file__ issues in notebooks

    # Enable caching and set cache directory
    cache_dir = os.path.join(BASE_DIR, "cache")
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    fastf1.Cache.enable_cache(cache_dir)

    # Load raw session data for telemetry
    telemetry_metrics = []
    for session_label in session_labels:
        logger.info(f"Loading {session_label} from cache or downloading for telemetry...")
        try:
            session = fastf1.get_session(year, round_number, session_label)
            session.load(laps=True, telemetry=True, weather=True)  # Load full session data without force_load
            laps = session.laps

            # Extract telemetry for a sample lap per driver (e.g., fastest lap)
            driver_metrics = {}
            for driver in laps['Driver'].unique():
                driver_laps = laps[laps['Driver'] == driver]
                fastest_lap = driver_laps.loc[driver_laps['LapTime'].idxmin()]
                if not pd.isna(fastest_lap['LapTime']):
                    telemetry = fastest_lap.get_telemetry()
                    # Metrics: Max Speed, % Time at Full Throttle, Avg Braking Intensity
                    max_speed = telemetry['Speed'].max()
                    throttle_time = (telemetry['Throttle'] == 100).mean() * 100  # % time at full throttle
                    braking_intensity = telemetry['Brake'].mean()  # Avg brake application (0-1)
                    driver_metrics[driver] = {
                        'Driver': driver,
                        'MaxSpeed': max_speed,
                        'ThrottleTime': throttle_time,
                        'BrakingIntensity': braking_intensity,
                        'Session': session_label
                    }

            # Convert to DataFrame and append
            session_df = pd.DataFrame.from_dict(driver_metrics, orient='index')
            telemetry_metrics.append(session_df)
        except Exception as e:
            logger.warning(f"Error loading {session_label}: {e}. Skipping session.")
            continue

    if not telemetry_metrics:
        logger.error("No telemetry data available to process.")
        return pd.DataFrame()

    # Combine and average metrics across sessions
    all_telemetry = pd.concat(telemetry_metrics, ignore_index=True)
    overall_telemetry = all_telemetry.groupby('Driver').mean(numeric_only=True).reset_index()

    # Merge with team info (from FP3 raw data)
    fp3_file = os.path.join(BASE_DIR, data_dir, f"{year}_R{round_number}_Practice_3_laps.csv")  # Updated to match Step 3 naming
    if os.path.exists(fp3_file):
        fp3_data = pd.read_csv(fp3_file)
        team_info = fp3_data[['Driver', 'Team']].drop_duplicates()
        overall_telemetry = overall_telemetry.merge(team_info, on='Driver', how='left')
    else:
        logger.warning(f"Warning: {fp3_file} not found. Team info will be incomplete.")
        overall_telemetry['Team'] = 'Unknown'  # Default value if no team data

    # Display table
    print("\nTelemetry Metrics (Top 10 by Max Speed):")
    display(overall_telemetry.sort_values('MaxSpeed', ascending=False).head(10))

    # Visualization with bar charts
    plt.figure(figsize=(12, 8))
    metrics = ['MaxSpeed', 'ThrottleTime', 'BrakingIntensity']
    titles = ['Max Speed (km/h)', '% Time at Full Throttle', 'Avg Braking Intensity (0-1)']
    colors = ['blue', 'orange', 'green']

    for i, (metric, title, color) in enumerate(zip(metrics, titles, colors), 1):
        plt.subplot(3, 1, i)
        # For ThrottleTime, filter out drivers with 0% (invalid data)
        if metric == 'ThrottleTime':
            plot_data = overall_telemetry[overall_telemetry['ThrottleTime'] > 0]
        else:
            plot_data = overall_telemetry
        sorted_data = plot_data.sort_values(metric, ascending=False)
        plt.bar(sorted_data['Driver'], sorted_data[metric], color=color)

        # Adjust y-axis limits for better readability
        y_min = sorted_data[metric].min()
        y_max = sorted_data[metric].max()
        y_range = y_max - y_min
        plt.ylim(y_min - 0.05 * y_range, y_max + 0.05 * y_range)

        plt.xlabel('Driver')
        plt.ylabel(title)
        plt.title(f'Telemetry Metrics - {metric} ({year} {event_name})')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Evaluation and Conclusion
    print("\n=== Evaluation and Conclusion ===")
    print(f"Telemetry metrics for {year} Round {round_number} - {event_name} provide insights into car setup and driver behavior:")

    # Identify top performers and outliers
    top_speed_driver = overall_telemetry.loc[overall_telemetry['MaxSpeed'].idxmax(), 'Driver']
    top_speed_value = overall_telemetry['MaxSpeed'].max()
    low_speed_driver = overall_telemetry.loc[overall_telemetry['MaxSpeed'].idxmin(), 'Driver']
    low_speed_value = overall_telemetry['MaxSpeed'].min()

    # Filter out invalid ThrottleTime for evaluation
    valid_throttle = overall_telemetry[overall_telemetry['ThrottleTime'] > 0]
    if not valid_throttle.empty:
        top_throttle_driver = valid_throttle.loc[valid_throttle['ThrottleTime'].idxmax(), 'Driver']
        top_throttle_value = valid_throttle['ThrottleTime'].max()
        low_throttle_driver = valid_throttle.loc[valid_throttle['ThrottleTime'].idxmin(), 'Driver']
        low_throttle_value = valid_throttle['ThrottleTime'].min()
    else:
        top_throttle_driver = "N/A"
        top_throttle_value = "N/A"
        low_throttle_driver = "N/A"
        low_throttle_value = "N/A"

    top_braking_driver = overall_telemetry.loc[overall_telemetry['BrakingIntensity'].idxmax(), 'Driver']
    top_braking_value = overall_telemetry['BrakingIntensity'].max()
    low_braking_driver = overall_telemetry.loc[overall_telemetry['BrakingIntensity'].idxmin(), 'Driver']
    low_braking_value = overall_telemetry['BrakingIntensity'].min()

    # Generic insights with dynamic data
    print("- **Max Speed**: Indicates straight-line performance. Higher values suggest setups optimized for speed, which can be advantageous on tracks with long straights or high-speed sections.")
    print(f"  - Top Performer: {top_speed_driver} with {top_speed_value:.2f} km/h, indicating strong straight-line speed.")
    print(f"  - Lowest: {low_speed_driver} with {low_speed_value:.2f} km/h, possibly prioritizing downforce for cornering over top speed.")

    print("- **Throttle Time**: Reflects driving style or setup focus. High percentages indicate aggressive driving or setups designed for sustained high-speed performance, while low values may suggest a focus on technical corners or conservative driving.")
    if not valid_throttle.empty:
        print(f"  - Top Performer: {top_throttle_driver} with {top_throttle_value:.2f}% time at full throttle, suggesting an aggressive approach or high-speed setup.")
        print(f"  - Lowest: {low_throttle_driver} with {low_throttle_value:.2f}% time at full throttle, which may indicate a setup favoring cornering or potential sandbagging.")
    else:
        print("  - Note: Throttle time data is missing or invalid for all drivers, which may indicate telemetry issues. Investigate data collection for accuracy.")

    # Note missing throttle time drivers
    missing_throttle = overall_telemetry[overall_telemetry['ThrottleTime'] == 0]['Driver'].tolist()
    if missing_throttle:
        print(f"  - Missing Throttle Time: The following drivers have 0% throttle time, which may indicate telemetry errors: {', '.join(missing_throttle)}. These drivers are excluded from the throttle time chart.")

    print("- **Braking Intensity**: Measures average brake application (0-1). Higher values indicate heavier braking, often associated with late-braking techniques, while lower values suggest smoother driving, which can be advantageous in technical sections.")
    print(f"  - Top Performer: {top_braking_driver} with an intensity of {top_braking_value:.3f}, potentially excelling in late braking into corners.")
    print(f"  - Lowest: {low_braking_driver} with an intensity of {low_braking_value:.3f}, indicating a smoother braking style that may preserve tyre life.")

    # Combined interpretation
    print("\n**Combined Interpretation**:")
    print("Drivers with high max speed and throttle time are likely to perform well in qualifying, especially on tracks favoring straight-line speed. Those with balanced braking intensity may excel in races, particularly on circuits with technical corners where tyre management and cornering stability are key.")
    print("Look for outliers: Drivers with unusually low throttle time or high braking intensity may be sandbagging or using setups that prioritize race pace over qualifying performance. Correlate these metrics with lap times (Step 5) and sandbagging analysis (Step 7) to identify potential hidden pace.")

    # Save table
    output_file = os.path.join(BASE_DIR, data_dir, f"{year}_R{round_number}_telemetry_metrics.csv")
    overall_telemetry.to_csv(output_file, index=False)
    print(f"Saved telemetry metrics to {output_file}")

    return overall_telemetry

# Example usage with dynamic target_info
telemetry_df = analyze_telemetry_metrics(target_info)

# Why: Telemetry reveals car setup (speed, downforce) and driver aggression, key for pole and race.
# How to Read: Higher Max Speed = straight-line strength. High Throttle = aggressive driving. Moderate Braking = technical skill.
# Expected Inputs: target_info with year, round, session_types, event_name
# Expected Outputs: Table of metrics, bar charts, evaluation, saved CSV
# Data Exported: {year}_R{round}_telemetry_metrics.csv in raw_data/{year}/R{round_number}/ directory

In [ ]:
# Step 7: Sandbagging Analysis (Updated to Use Driver Performance Data)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import os
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

if not logger.handlers:
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

def investigate_sandbagging(config):
    """
    Detect teams hiding pace by comparing lap times to telemetry metrics.

    Args:
        config (dict): Configuration dictionary containing:
            - 'year': The race year (e.g., 2024)
            - 'round': The round number (e.g., 3)
            - 'data_dir': Directory for raw data (default: "raw_data")
            - 'sandbag_threshold': Discrepancy threshold for sandbagging (default: 0.15 seconds)

    Returns:
        pd.DataFrame: Sandbagging analysis results
    """
    year = config.get('year')
    round_number = config.get('round')  # Matches target_info
    data_dir = os.path.join("raw_data", str(year), f"R{round_number}")  # Updated to match project structure
    sandbag_threshold = config.get('sandbag_threshold', 0.15)

    BASE_DIR = os.getcwd()  # Jupyter-friendly, avoids __file__ issues
    logger.info(f"Base directory: {BASE_DIR}")

    # Load data
    driver_performance_file = os.path.join(BASE_DIR, data_dir, f"{year}_R{round_number}_driver_performance.csv")
    telemetry_file = os.path.join(BASE_DIR, data_dir, f"{year}_R{round_number}_telemetry_metrics.csv")
    if not os.path.exists(driver_performance_file):
        logger.error(f"Driver performance file not found: {driver_performance_file}")
        return pd.DataFrame()
    if not os.path.exists(telemetry_file):
        logger.error(f"Telemetry metrics file not found: {telemetry_file}")
        return pd.DataFrame()

    driver_performance = pd.read_csv(driver_performance_file)
    telemetry_metrics = pd.read_csv(telemetry_file)

    # Merge data
    sandbag_analysis = driver_performance[['Driver', 'FastestLapTime', 'Team']].merge(
        telemetry_metrics[['Driver', 'MaxSpeed', 'ThrottleTime']], on='Driver', how='left'
    )
    # Check for NaN values in telemetry metrics
    if sandbag_analysis[['MaxSpeed', 'ThrottleTime']].isna().any().any():
        logger.warning("NaN values found in telemetry metrics. Filling with median values.")
        sandbag_analysis['MaxSpeed'] = sandbag_analysis['MaxSpeed'].fillna(telemetry_metrics['MaxSpeed'].median())
        sandbag_analysis['ThrottleTime'] = sandbag_analysis['ThrottleTime'].fillna(telemetry_metrics['ThrottleTime'].median())

    sandbag_analysis['ExpectedLapTime'] = driver_performance['Sector1Time'] + driver_performance['Sector2Time'] + driver_performance['Sector3Time']
    sandbag_analysis['LapTimeDiscrepancy'] = sandbag_analysis['FastestLapTime'] - sandbag_analysis['ExpectedLapTime']

    # Predict FastestLapTime from MaxSpeed using linear regression
    X = sandbag_analysis[['MaxSpeed']].values
    y = sandbag_analysis['FastestLapTime'].values
    reg = LinearRegression().fit(X, y)
    sandbag_analysis['PredictedLapTime'] = reg.predict(X)
    sandbag_analysis['PredictionDiscrepancy'] = sandbag_analysis['FastestLapTime'] - sandbag_analysis['PredictedLapTime']
    sandbag_analysis['SpeedRank'] = sandbag_analysis['MaxSpeed'].rank(ascending=False)
    sandbag_analysis['LapTimeRank'] = sandbag_analysis['FastestLapTime'].rank()

    # New sandbagging logic: Flag drivers with high MaxSpeed (above mean) and FastestLapTime exceeding PredictedLapTime + threshold
    mean_max_speed = sandbag_analysis['MaxSpeed'].mean()
    sandbag_analysis['SandbagFlag'] = (sandbag_analysis['MaxSpeed'] > mean_max_speed) & (sandbag_analysis['PredictionDiscrepancy'] > sandbag_threshold)

    # Display potential sandbaggers
    print("\nPotential Sandbaggers (Sorted by PredictionDiscrepancy):")
    display(sandbag_analysis[sandbag_analysis['SandbagFlag']].sort_values('PredictionDiscrepancy', ascending=False))

    # Visualization: Scatter plot of FastestLapTime vs. MaxSpeed with sandbaggers highlighted
    plt.figure(figsize=(12, 6))
    colors = ['blue' if not flag else 'red' for flag in sandbag_analysis['SandbagFlag']]
    scatter = plt.scatter(sandbag_analysis['FastestLapTime'], sandbag_analysis['MaxSpeed'], c=colors, label='Drivers', alpha=0.6)
    plt.axhline(y=mean_max_speed, color='gray', linestyle='--', label='Mean Max Speed')
    plt.axvline(x=sandbag_analysis['PredictedLapTime'].mean() + sandbag_threshold, color='green', linestyle='--', label=f'Threshold (Predicted + {sandbag_threshold}s)')
    # Plot regression line
    x_range = np.array([[sandbag_analysis['MaxSpeed'].min()], [sandbag_analysis['MaxSpeed'].max()]])
    y_pred = reg.predict(x_range)
    plt.plot(y_pred, x_range, color='orange', linestyle='-', label='Regression Line')
    for i, row in sandbag_analysis.iterrows():
        plt.text(row['FastestLapTime'], row['MaxSpeed'], row['Driver'], fontsize=8, ha='right', va='bottom')
        if row['SandbagFlag']:
            plt.annotate(row['Driver'], (row['FastestLapTime'], row['MaxSpeed']), xytext=(5, 5), textcoords='offset points', fontsize=10, color='red', weight='bold')
    plt.xlabel('Fastest Lap Time (seconds)')
    plt.ylabel('Max Speed (km/h)')
    plt.title(f'Sandbagging Analysis: Fastest Lap Time vs. Max Speed ({year} Round {round_number})')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Save results
    output_file = os.path.join(BASE_DIR, data_dir, f"{year}_R{round_number}_sandbag_analysis.csv")
    sandbag_analysis.to_csv(output_file, index=False)
    print(f"Saved sandbag analysis to {output_file}")

    return sandbag_analysis

# Example usage with dynamic target_info
sandbag_df = investigate_sandbagging(target_info)

# Why: Detects teams hiding pace by comparing lap times to telemetry, aiding race prediction.
# How to Read: High PredictionDiscrepancy with high MaxSpeed flags sandbagging. Red dots on scatter plot highlight suspects.
# Expected Inputs: driver_performance.csv, telemetry_metrics.csv from prior steps
# Expected Outputs: Table of sandbaggers, scatter plot, saved CSV
# Data Exported: {year}_R{round_number}_sandbag_analysis.csv in raw_data/{year}/R{round_number}/ directory

In [ ]:
# Step 8 - Evaluate Track Evolution and Weather Impact
import pandas as pd
import matplotlib.pyplot as plt
import fastf1
import numpy as np
from scipy.stats import pearsonr

def evaluate_track_evolution(target_info, cleaned_data):
    year = target_info['year']
    round_number = target_info['round']
    session_labels = target_info['session_types']
    weather_metric = target_info.get('weather_metric', 'TrackTemp')
    session_label_map = {
        'FP1': 'Practice 1', 'FP2': 'Practice 2', 'FP3': 'Practice 3',
        'Sprint Qualifying': 'Sprint Qualifying', 'Sprint': 'Sprint'
    }
    session_labels = [session_label_map.get(label, label) for label in session_labels]
    event_name = target_info.get('event_name', 'Unknown Event')
    data_dir = os.path.join(os.getcwd(), "raw_data", str(year), f"R{round_number}")

    # Lap stats from cleaned_data
    lap_stats = {}
    for session_label in session_labels:
        df = cleaned_data.get(session_label, pd.DataFrame())
        if df.empty:
            print(f"Warning: No lap data for {session_label}, skipping.")
            continue
        if pd.api.types.is_string_dtype(df['LapTime']):
            df['LapTime'] = pd.to_timedelta(df['LapTime'])
        lap_times = df['LapTime'].dt.total_seconds().dropna()
        if len(lap_times) == 0:
            print(f"Warning: No valid lap times for {session_label}, skipping.")
            continue
        lap_stats[session_label] = {
            'mean': lap_times.mean(),
            'std': lap_times.std(),
            'count': len(lap_times)
        }
        print(f"Average lap time for {session_label}: {lap_stats[session_label]['mean']:.3f} seconds (±{lap_stats[session_label]['std']:.3f})")

    # Weather data
    weather_data = {}
    for session_label in session_labels:
        print(f"Loading weather for {session_label} from FastF1...")
        try:
            session = fastf1.get_session(year, round_number, session_label)
            session.load(weather=True, laps=False)
            weather = session.weather_data
            print(f"Available weather columns for {session_label}: {weather.columns.tolist()}")
            available_columns = [col for col in ['AirTemp', 'TrackTemp', 'Humidity', 'Rainfall'] if col in weather.columns]
            if available_columns:
                weather_data[session_label] = weather[available_columns]
            else:
                print(f"No valid weather data for {session_label}, skipping weather analysis.")
                weather_data[session_label] = pd.DataFrame()
        except Exception as e:
            print(f"Failed to load {session_label} weather data: {e}")
            weather_data[session_label] = pd.DataFrame()

    avg_weather = {session: weather.mean() for session, weather in weather_data.items() if not weather.empty}

    # Tables
    lap_time_df = pd.DataFrame([
        {'Session': session, 'AvgLapTime': stats['mean'], 'StdLapTime': stats['std'], 'LapCount': stats['count']}
        for session, stats in lap_stats.items()
    ])
    print("\nAverage Lap Times Across Sessions:")
    display(lap_time_df)

    weather_list = [
        {'Session': session, 'WeatherType': col, 'Value': value}
        for session, stats in avg_weather.items() if not stats.empty
        for col, value in stats.to_dict().items()
    ]
    weather_df = pd.DataFrame(weather_list) if weather_list else pd.DataFrame()
    if not weather_df.empty:
        print("\nAverage Weather Conditions Across Sessions:")
        display(weather_df.pivot(index='Session', columns='WeatherType', values='Value'))

    # Correlation
    if not weather_df.empty and weather_metric in weather_df.pivot(index='Session', columns='WeatherType', values='Value').columns:
        weather_values = weather_df[weather_df['WeatherType'] == weather_metric].set_index('Session')['Value']
        lap_times = lap_time_df.set_index('Session')['AvgLapTime']
        common_sessions = weather_values.index.intersection(lap_times.index)
        if len(common_sessions) > 1:
            weather_values = weather_values[common_sessions]
            lap_times = lap_times[common_sessions]
            correlation, _ = pearsonr(weather_values, lap_times)
            print(f"\nCorrelation between {weather_metric} and AvgLapTime: {correlation:.3f}")
        else:
            print(f"\nNot enough overlapping sessions for correlation analysis with {weather_metric}.")
    else:
        print(f"\n{weather_metric} data not available for correlation analysis.")

    # Plot
    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.plot(lap_time_df['Session'], lap_time_df['AvgLapTime'], marker='o', color='blue', label='Avg Lap Time (s)')
    ax1.fill_between(lap_time_df['Session'], lap_time_df['AvgLapTime'] - lap_time_df['StdLapTime'] / 2,
                     lap_time_df['AvgLapTime'] + lap_time_df['StdLapTime'] / 2, color='blue', alpha=0.2, label='Lap Time Std Dev (±0.5σ)')
    ax1.set_xlabel('Session')
    ax1.set_ylabel('Average Lap Time (seconds)', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.set_xticks(lap_time_df['Session'])
    ax1.set_xticklabels(['P1', 'P2', 'P3'], rotation=45)

    if not weather_df.empty and weather_metric in weather_df.pivot(index='Session', columns='WeatherType', values='Value').columns:
        ax2 = ax1.twinx()
        weather_values_df = weather_df[weather_df['WeatherType'] == weather_metric].set_index('Session')['Value']
        weather_values_df = weather_values_df.reindex(lap_time_df['Session'])
        ax2.plot(lap_time_df['Session'], weather_values_df, marker='o', color='orange', label=f'{weather_metric} (°C)')
        ax2.set_ylabel(f'{weather_metric} (°C)', color='orange')
        ax2.tick_params(axis='y', labelcolor='orange')
        ax2.set_ylim(weather_values_df.min() - 2, weather_values_df.max() + 2)

    lines, labels = ax1.get_legend_handles_labels()
    if 'ax2' in locals():
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines + lines2, labels + labels2, loc='upper left')
    else:
        ax1.legend(loc='upper left')

    plt.title(f'Track Evolution - {year} {event_name} (Round {round_number})')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Save
    output_file = os.path.join(data_dir, f"{year}_R{round_number}_track_evolution.csv")
    lap_time_df.to_csv(output_file, index=False)
    print(f"Saved track evolution to {output_file}")
    return lap_time_df

# Standalone execution for debugging (runs only if cell is executed directly)
if 'target_info' in globals() and 'cleaned_data' in globals():
    track_evolution_df = evaluate_track_evolution(target_info, cleaned_data)
else:
    print("Warning: target_info and/or cleaned_data not found. Run Cells 2 and 4 first for standalone execution.")

In [ ]:
# Step 9 - Analyze Track Characteristics
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import fastf1
import os
from scipy.stats import zscore

def analyze_track_characteristics(target_info, cleaned_data):
    year = target_info['year']
    round_number = target_info['round']
    event_name = target_info.get('event_name', 'Unknown Event')
    data_dir = os.path.join(os.getcwd(), "raw_data", str(year), f"R{round_number}")

    all_laps = pd.concat([cleaned_data[session] for session in cleaned_data.keys()], ignore_index=True)
    required_columns = ['Driver', 'Team', 'SpeedST', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']
    if not all(col in all_laps.columns for col in required_columns):
        missing_cols = [col for col in required_columns if col not in all_laps.columns]
        print(f"Warning: Missing columns in cleaned_data: {missing_cols}. Using available data.")
        all_laps = all_laps.dropna(subset=[col for col in required_columns if col in all_laps.columns])

    track_analysis = all_laps.groupby(['Driver', 'Team']).agg({
        'SpeedST': 'max',
        'Sector1Time': 'mean',
        'Sector2Time': 'mean',
        'Sector3Time': 'mean',
        'LapTime': 'mean'
    }).reset_index()

    for col in ['Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']:
        if pd.api.types.is_string_dtype(track_analysis[col]):
            track_analysis[col] = pd.to_timedelta(track_analysis[col])
        track_analysis[col] = track_analysis[col].dt.total_seconds()

    try:
        session = fastf1.get_session(year, round_number, 'Practice 3')
        session.load(laps=True, telemetry=True, weather=False)
        circuit_info = session.get_circuit_info()
        lap_length = circuit_info.length / 1000 if hasattr(circuit_info, 'length') else target_info.get('track_length', 6.174)
        drs_zones = len(circuit_info.drs_zones) if hasattr(circuit_info, 'drs_zones') and circuit_info.drs_zones else 3
    except Exception as e:
        print(f"Error fetching circuit info: {e}")
        lap_length = target_info.get('track_length', 6.174)
        drs_zones = 3

    avg_sector_speed = all_laps['SpeedST'].mean()
    max_sector_speed = all_laps['SpeedST'].max()
    downforce_requirement = 'Low' if avg_sector_speed > 330 else 'Medium-High' if avg_sector_speed > 310 else 'High'
    overtaking_difficulty = 'Easy' if drs_zones > 2 else 'Moderate' if drs_zones == 2 else 'Hard'
    try:
        driver_performance = pd.read_csv(os.path.join(data_dir, f"{year}_R{round_number}_driver_performance.csv"))
        avg_tyre_degradation = driver_performance['DegradationSlope'].mean()
        tyre_degradation = 'High' if avg_tyre_degradation > 10 else 'Moderate' if avg_tyre_degradation > 7 else 'Low'
    except Exception as e:
        print(f"Error loading driver performance data: {e}")
        tyre_degradation = 'Moderate'
    suits_fast_cars = 'Yes' if max_sector_speed > 330 else 'No'

    track_characteristics = {
        'Track': event_name,
        'DownforceRequirement': downforce_requirement,
        'OvertakingDifficulty': overtaking_difficulty,
        'DRSZones': drs_zones,
        'TyreDegradation': tyre_degradation,
        'LapLength': lap_length,
        'SuitsFastCars': suits_fast_cars
    }

    track_analysis['StraightSpeedPotential'] = track_analysis['SpeedST']
    track_analysis['CorneringPotential'] = (track_analysis['Sector1Time'] + track_analysis['Sector2Time'] + track_analysis['Sector3Time']) / 3
    track_analysis['AvgSpeed'] = (lap_length * 3600) / track_analysis['LapTime']
    straight_line_ratio = track_analysis['StraightSpeedPotential'].mean() / track_analysis['CorneringPotential'].mean()
    weight_straight = straight_line_ratio / (straight_line_ratio + 1) if suits_fast_cars == 'Yes' else 0.4
    weight_cornering = 1 - weight_straight
    track_analysis['StraightZScore'] = zscore(track_analysis['StraightSpeedPotential'])
    track_analysis['CorneringZScore'] = -zscore(track_analysis['CorneringPotential'])
    track_analysis['OverallFit'] = (weight_straight * track_analysis['StraightZScore'] + weight_cornering * track_analysis['CorneringZScore'])

    track_analysis = track_analysis.sort_values('OverallFit', ascending=False)
    track_analysis['FitRank'] = track_analysis['OverallFit'].rank(ascending=False)

    print(f"\nTrack Characteristics Analysis for {event_name}:")
    track_char_df = pd.DataFrame(track_characteristics, index=[0])
    display(track_char_df)
    print(f"\nDriver Fit to Track Characteristics (Top 10) - {event_name}:")
    display(track_analysis[['Driver', 'Team', 'StraightSpeedPotential', 'CorneringPotential', 'AvgSpeed', 'OverallFit', 'FitRank']].head(10))

    plt.figure(figsize=(12, 6))
    scatter = plt.scatter(track_analysis['StraightSpeedPotential'], track_analysis['CorneringPotential'],
                          c=track_analysis['FitRank'], cmap='viridis')
    plt.colorbar(scatter, label='Fit Rank')
    for i, row in track_analysis.iterrows():
        plt.text(row['StraightSpeedPotential'], row['CorneringPotential'], row['Driver'], fontsize=8, ha='right', va='bottom')
    plt.xlabel('Straight Speed Potential (km/h)')
    plt.ylabel('Cornering Potential (seconds)')
    plt.title(f'Track Fit Analysis - {year} {event_name} (Round {round_number})')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 6))
    scatter = plt.scatter(track_analysis['StraightSpeedPotential'], track_analysis['AvgSpeed'],
                          c=track_analysis['FitRank'], cmap='viridis')
    plt.colorbar(scatter, label='Fit Rank')
    for i, row in track_analysis.iterrows():
        plt.text(row['StraightSpeedPotential'], row['AvgSpeed'], row['Driver'], fontsize=8, ha='right', va='bottom')
    plt.xlabel('Max Straight Speed (km/h)')
    plt.ylabel('Average Speed (km/h)')
    plt.title(f'Aeromap - {year} {event_name} (Round {round_number})')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    output_file = os.path.join(data_dir, f"{year}_R{round_number}_track_characteristics.csv")
    driver_file = os.path.join(data_dir, f"{year}_R{round_number}_track_characteristics_drivers.csv")
    track_char_df.to_csv(output_file, index=False)
    track_analysis.to_csv(driver_file, index=False)
    print(f"Saved track characteristics to {output_file}")
    print(f"Saved driver fit data to {driver_file}")
    return track_analysis  # Return driver fit data as primary output

# Standalone execution for debugging (runs only if cell is executed directly)
if 'target_info' in globals() and 'cleaned_data' in globals():
    track_char_drivers_df = analyze_track_characteristics(target_info, cleaned_data)
else:
    print("Warning: target_info and/or cleaned_data not found. Run Cells 2 and 4 first for standalone execution.")

In [ ]:
# Step 10: Race Strategy and Final Prediction
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import fastf1

def predict_race_outcomes(config, driver_performance_df, telemetry_metrics_df, sandbag_analysis_df, track_evolution_df, track_char_drivers_df, pit_stop_loss_default=25, overtaking_penalty=0.5):
    year = config.get('year')
    round_number = config.get('round')
    data_dir = config.get('data_dir', "raw_data")
    race_laps = config.get('race_laps', 50)
    event_name = config.get('event_name', "Unknown Event")

    BASE_DIR = os.getcwd()
    round_data_dir = os.path.join(BASE_DIR, data_dir, str(year), f"R{round_number}")
    os.makedirs(round_data_dir, exist_ok=True)

    # Load dynamic pit_stop_loss
    pit_stop_file = os.path.join(BASE_DIR, "pit_stop_loss_by_track.csv")
    if os.path.exists(pit_stop_file):
        pit_stop_df = pd.read_csv(pit_stop_file)
        if event_name in pit_stop_df['Track'].values:
            pit_stop_loss = pit_stop_df.loc[pit_stop_df['Track'] == event_name, 'PitStopLoss'].values[0]
            print(f"Loaded pit_stop_loss from file: {pit_stop_loss:.2f}s for {event_name}")
        else:
            pit_stop_loss = pit_stop_loss_default
            print(f"No pit_stop_loss found for {event_name} in file, using default: {pit_stop_loss}s")
    else:
        pit_stop_loss = pit_stop_loss_default
        print(f"Pit stop loss file not found, using default: {pit_stop_loss}s")

    # Handle NaN in driver_performance_df
    driver_performance_df['BasePace'] = driver_performance_df['BasePace'].fillna(driver_performance_df['FastestLapTime'] + 5)
    driver_performance_df['DegradationSlope'] = driver_performance_df['DegradationSlope'].fillna(7.0)
    driver_performance_df['LapTimeVar'] = driver_performance_df['LapTimeVar'].fillna(driver_performance_df['BasePace'] * 0.01)

    # Qualifying Prediction
    quali_prediction = driver_performance_df[['Driver', 'FastestLapTime', 'Team']].copy()
    quali_prediction = quali_prediction.rename(columns={'FastestLapTime': 'LapTime'})
    quali_prediction['ExpectedLapTime'] = driver_performance_df['Sector1Time'] + driver_performance_df['Sector2Time'] + driver_performance_df['Sector3Time']
    quali_prediction = quali_prediction.merge(sandbag_analysis_df[['Driver', 'LapTimeDiscrepancy', 'SandbagFlag']], on='Driver', how='left')
    quali_prediction = quali_prediction.merge(track_char_drivers_df[['Driver', 'OverallFit']], on='Driver', how='left')
    quali_prediction['AdjustedLapTime'] = quali_prediction.apply(
        lambda row: row['ExpectedLapTime'] if (row.get('SandbagFlag', False) or (row.get('OverallFit', 0) > 0.5 and row['LapTime'] > row['ExpectedLapTime'])) else row['LapTime'], axis=1
    )
    quali_prediction['QualiPosition'] = quali_prediction['AdjustedLapTime'].rank(method='min').astype(int)

    print("\nQualifying Prediction:")
    display(quali_prediction[['Driver', 'Team', 'LapTime', 'ExpectedLapTime', 'AdjustedLapTime', 'QualiPosition']].sort_values('QualiPosition'))

    # Race Strategy Simulation
    strategy_df = driver_performance_df.copy()
    strategy_df['QualiPace'] = quali_prediction['AdjustedLapTime']
    strategy_df['GridPosition'] = quali_prediction['QualiPosition']

    strategy_df['FuelPenalty'] = strategy_df['DegradationSlope'] * (race_laps / 2) / 100 * 2
    strategy_df['TrafficPenalty'] = (strategy_df['LapTimeVar'] / strategy_df['BasePace']) * overtaking_penalty * (race_laps / 50) * 5

    strategies = []
    for _, row in strategy_df.iterrows():
        driver = row['Driver']
        team = row['Team']
        base_pace = row['BasePace']
        deg_rate = row['DegradationSlope'] / 100
        fuel_penalty = row['FuelPenalty']
        traffic_penalty = row['TrafficPenalty'] if row['GridPosition'] > 5 else row['TrafficPenalty'] * 0.5
        grid_pos = row['GridPosition']
        quali_pace = row['QualiPace']

        start_compound = 'MEDIUM' if grid_pos <= 10 else 'SOFT'
        pos_penalty = (grid_pos - 1) * (quali_pace - strategy_df['QualiPace'].min()) / 5
        adjusted_pace = base_pace + pos_penalty + fuel_penalty + traffic_penalty

        max_stint = 27
        if start_compound == 'SOFT':
            first_stint = min(max_stint, max(5, 15 / (max(deg_rate * 3, 0.07))))
        else:
            first_stint = min(max_stint, max(7, 20 / (max(deg_rate * 2, 0.07))))
        second_compound = 'HARD'
        second_stint = race_laps - first_stint

        if first_stint > 7:
            first_stint = 7
            second_stint = race_laps - first_stint

        one_stop_pace = ((adjusted_pace + deg_rate * (first_stint / 2)) * first_stint +
                         (adjusted_pace + deg_rate * (second_stint / 2)) * second_stint + pit_stop_loss)
        one_stop_stints = [first_stint, second_stint]
        one_stop_compounds = [start_compound, second_compound]

        if deg_rate * race_laps > 1.0:
            first_stint = min(max_stint, race_laps / 3)
            second_compound = 'MEDIUM' if start_compound != 'MEDIUM' else 'SOFT'
            second_stint_length = min(max_stint, (race_laps - first_stint) / 2)
            third_compound = 'HARD'
            third_stint = race_laps - first_stint - second_stint_length
            if third_stint > 0:
                two_stop_pace = ((adjusted_pace + deg_rate * (first_stint / 2)) * first_stint +
                                 (adjusted_pace + deg_rate * (second_stint_length / 2)) * second_stint_length +
                                 (adjusted_pace + deg_rate * (third_stint / 2)) * third_stint + 2 * pit_stop_loss)
                two_stop_stints = [first_stint, second_stint_length, third_stint]
                two_stop_compounds = [start_compound, second_compound, third_compound]
            else:
                two_stop_pace = one_stop_pace + pit_stop_loss
                two_stop_stints = one_stop_stints
                two_stop_compounds = one_stop_compounds
        else:
            two_stop_pace = float('inf')
            two_stop_stints = []
            two_stop_compounds = []

        best_time = min(one_stop_pace, two_stop_pace)
        strategy = 'One-Stop' if one_stop_pace <= two_stop_pace else 'Two-Stop'
        stint_lengths = one_stop_stints if strategy == 'One-Stop' else two_stop_stints
        compounds = one_stop_compounds if strategy == 'One-Stop' else two_stop_compounds

        strategies.append({
            'Driver': driver,
            'Team': team,
            'GridPosition': grid_pos,
            'QualiPace': quali_pace,
            'BasePace': base_pace,
            'LapTimeVar': row['LapTimeVar'],
            'DegradationSlope': row['DegradationSlope'],
            'FuelPenalty': fuel_penalty,
            'TrafficPenalty': traffic_penalty,
            'CompoundStrategy': '-'.join(compounds),
            'StintLengths': stint_lengths,
            'TotalRaceTime': best_time
        })

    race_prediction = pd.DataFrame(strategies)

    race_prediction = race_prediction.merge(telemetry_metrics_df[['Driver', 'MaxSpeed', 'ThrottleTime']], on='Driver', how='left')
    race_prediction = race_prediction.merge(track_char_drivers_df[['Driver', 'OverallFit', 'AvgSpeed']], on='Driver', how='left')

    p2_lap_time = track_evolution_df[track_evolution_df['Session'] == 'Practice 2']['AvgLapTime'].iloc[0]
    p3_lap_time = track_evolution_df[track_evolution_df['Session'] == 'Practice 3']['AvgLapTime'].iloc[0]
    track_evolution_factor = (p3_lap_time - p2_lap_time) / p2_lap_time

    chaos_factor = (race_prediction['LapTimeVar'].mean() / race_prediction['BasePace'].mean()) * race_prediction['LapTimeVar'].max() / 20
    quali_gaps = race_prediction['QualiPace'].max() - race_prediction['QualiPace'].min()
    overtaking_adjustment = (quali_gaps / (race_prediction['GridPosition'].max() - 1)) * 0.2 if race_prediction['GridPosition'].max() > 1 else 0.2
    race_load_factor = (race_prediction['BasePace'].mean() - race_prediction['QualiPace'].mean()) / (race_laps / 10) * 1.5

    race_prediction['RaceLoad'] = race_load_factor * (race_prediction['LapTimeVar'] / race_prediction['BasePace']) * (race_laps / 2)
    race_prediction['TrackEvolutionAdjustment'] = track_evolution_factor * race_prediction['TotalRaceTime'] * (race_prediction['DegradationSlope'] / 100)
    race_prediction['ConsistencyAdjustment'] = race_prediction['LapTimeVar'] / race_prediction['BasePace'] * 10
    race_prediction['AdjustedRaceTime'] = race_prediction['TotalRaceTime'] + race_prediction['RaceLoad'] + race_prediction['TrackEvolutionAdjustment'] + race_prediction['ConsistencyAdjustment']
    race_prediction['SpeedAdjustment'] = (race_prediction['MaxSpeed'] / race_prediction['MaxSpeed'].max()) * 3
    race_prediction['FitAdjustment'] = (1 - race_prediction['OverallFit'].rank() / len(race_prediction)) * 10
    race_prediction['AvgSpeedAdjustment'] = (race_prediction['AvgSpeed'] / race_prediction['AvgSpeed'].max()) * 20
    race_prediction['AdjustedRaceTime'] = race_prediction['AdjustedRaceTime'] - race_prediction['SpeedAdjustment'] - race_prediction['FitAdjustment'] - race_prediction['AvgSpeedAdjustment']

    for idx, row in race_prediction.iterrows():
        grid_pos = row['GridPosition']
        if grid_pos > 10:
            race_prediction.at[idx, 'AdjustedRaceTime'] += chaos_factor * (grid_pos - 10) / 10
        pos_diff = max(0, grid_pos - 5)
        race_prediction.at[idx, 'AdjustedRaceTime'] += pos_diff * overtaking_adjustment

    race_prediction['FinalPosition'] = race_prediction['AdjustedRaceTime'].rank(method='min').astype(int)

    print("\nFinal Race Prediction:")
    display(race_prediction[['Driver', 'Team', 'GridPosition', 'TotalRaceTime', 'AdjustedRaceTime', 'FinalPosition']].sort_values('FinalPosition'))

    # Fastest Lap Prediction
    fastest_laps = []
    for session_label in ['Practice 1', 'Practice 2', 'Practice 3']:
        try:
            session = fastf1.get_session(year, round_number, session_label)
            session.load(laps=True)
            laps = session.laps
            for driver in laps['Driver'].unique():
                driver_laps = laps[laps['Driver'] == driver]
                fastest_lap = driver_laps.loc[driver_laps['LapTime'].idxmin()]
                if not pd.isna(fastest_lap['LapTime']):
                    fastest_laps.append({
                        'Driver': driver,
                        'FastestLapTime': fastest_lap['LapTime'].total_seconds(),
                        'Session': session_label
                    })
        except Exception as e:
            print(f"Error loading {session_label} for fastest lap prediction: {e}")
            continue

    if fastest_laps:
        fastest_lap_df = pd.DataFrame(fastest_laps)
        fastest_lap_pred = fastest_lap_df.loc[fastest_lap_df['FastestLapTime'].idxmin(), 'Driver']
        print(f"\nPredicted Fastest Lap Driver: {fastest_lap_pred}")
    else:
        fastest_lap_pred = race_prediction.loc[race_prediction['AdjustedRaceTime'].idxmin(), 'Driver']
        print(f"\nNo fastest lap data available; fallback to race time prediction: {fastest_lap_pred}")

    # Save results
    quali_output_file = os.path.join(round_data_dir, f"{year}_R{round_number}_quali_prediction.csv")
    race_output_file = os.path.join(round_data_dir, f"{year}_R{round_number}_race_prediction.csv")
    quali_prediction.to_csv(quali_output_file, index=False)
    race_prediction.to_csv(race_output_file, index=False)
    print(f"Saved qualifying prediction to {quali_output_file}")
    print(f"Saved race prediction to {race_output_file}")

    return quali_prediction, race_prediction, fastest_lap_pred

# Standalone execution for debugging (runs only if required DataFrames are in scope)
if all(var in globals() for var in ['target_info', 'driver_performance_df', 'telemetry_metrics_df', 'sandbag_analysis_df', 'track_evolution_df', 'track_char_drivers_df']):
    quali_prediction_df, race_prediction_df, fastest_lap_pred = predict_race_outcomes(
        target_info, driver_performance_df, telemetry_metrics_df, sandbag_analysis_df, track_evolution_df, track_char_drivers_df
    )
else:
    print("Warning: Required variables (target_info, driver_performance_df, telemetry_metrics_df, sandbag_analysis_df, track_evolution_df, track_char_drivers_df) not found. Run Cells 2, 5, 6, 7, 8, and 9 first for standalone execution.")

In [ ]:
# Step 10.5: Comprehensive Pre-Qualifying Report
import pandas as pd
import os
import matplotlib.pyplot as plt
from fpdf import FPDF
import tempfile
import shutil

def generate_comprehensive_report(config, driver_performance_df, telemetry_metrics_df, sandbag_analysis_df, track_evolution_df, track_char_drivers_df, quali_prediction_df, race_prediction_df, fastest_lap_pred):
    year = config.get('year')
    round_number = config.get('round')
    event_name = config.get('event_name', "Unknown Event")
    data_dir = os.path.join("raw_data", str(year), f"R{round_number}")
    BASE_DIR = os.getcwd()
    cleaned_data_dir = os.path.join(BASE_DIR, data_dir, "cleaned_data")
    report_dir = os.path.join(BASE_DIR, "reports", str(year), f"R{round_number}")
    os.makedirs(report_dir, exist_ok=True)

    # Initialize PDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, f"Pre-Qualifying Report: {year} Round {round_number} - {event_name}", ln=True, align="C")
    pdf.ln(10)

    # Helper function to add DataFrame to PDF
    def add_df_to_pdf(df, title):
        pdf.set_font("Arial", "B", 12)
        pdf.cell(0, 10, title, ln=True)
        pdf.set_font("Arial", size=10)
        col_widths = [max([len(str(x)) for x in df[col].values] + [len(col)]) * 5 for col in df.columns]
        total_width = sum(col_widths)
        if total_width > 190:
            scale_factor = 190 / total_width
            col_widths = [w * scale_factor for w in col_widths]
        for i, row in df.iterrows():
            for j, val in enumerate(row):
                pdf.cell(col_widths[j], 10, str(val)[:20], border=1)
            pdf.ln()
        pdf.ln(5)

    # Helper function to add plot to PDF
    def add_plot_to_pdf(fig, title):
        pdf.set_font("Arial", "B", 12)
        pdf.cell(0, 10, title, ln=True)
        temp_dir = tempfile.mkdtemp()
        temp_file = os.path.join(temp_dir, f"{title.replace(' ', '_')}.png")
        try:
            fig.savefig(temp_file, format='png', bbox_inches='tight')
            pdf.image(temp_file, x=10, w=190)
        finally:
            plt.close(fig)
            shutil.rmtree(temp_dir, ignore_errors=True)
        pdf.ln(5)

    # Step 1: Event Info
    print("--- Step 1: Event Information ---")
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 1: Event Information", ln=True)
    info_text = (
        f"Year: {year}, Round: {round_number}, Event: {event_name}\n"
        f"Track Length: {config.get('track_length', 'N/A')} km\n"
        f"Average Speed: {config.get('avg_speed', 'N/A')} km/h\n"
        f"Downforce Level: {config.get('downforce_level', 'N/A')}"
    )
    print(info_text)
    pdf.set_font("Arial", size=10)
    pdf.multi_cell(0, 10, info_text)
    pdf.ln(5)

    # Step 2: Historical Data
    print("--- Step 2: Historical Data ---")
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 2: Historical Data", ln=True)
    hist_text = (
        f"Historical Winners: {config.get('historical_winners', 'N/A')}\n"
        f"Historical Poles: {config.get('historical_poles', 'N/A')}"
    )
    print(hist_text)
    pdf.set_font("Arial", size=10)
    pdf.multi_cell(0, 10, hist_text)
    pdf.ln(5)

    # Step 3: Session Data
    print("--- Step 3: Session Data ---")
    session_counts = {}
    for session in ['Practice 1', 'Practice 2', 'Practice 3']:
        try:
            session_file = os.path.join(BASE_DIR, data_dir, f"{year}_R{round_number}_{session.replace(' ', '_')}_laps.csv")
            session_data = pd.read_csv(session_file)
            session_counts[session] = len(session_data)
            print(f"{session} Lap Count: {session_counts[session]}")
        except Exception as e:
            print(f"Error loading {session} data: {e}")
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 3: Session Data", ln=True)
    pdf.set_font("Arial", size=10)
    for session, count in session_counts.items():
        pdf.cell(0, 10, f"{session} Lap Count: {count}", ln=True)
    pdf.ln(5)
    print()

    # Step 4: Data Cleaning Summary
    print("--- Step 4: Data Cleaning Summary ---")
    cleaned_counts = {}
    cleaned_data_files = [f"{year}_R{round_number}_Practice_{i}_cleaned_laps.csv" for i in range(1, 4)]
    for file in cleaned_data_files:
        try:
            cleaned_data = pd.read_csv(os.path.join(cleaned_data_dir, file))
            cleaned_counts[file] = len(cleaned_data)
            print(f"Cleaned {file}: {cleaned_counts[file]} laps")
        except Exception as e:
            print(f"Error loading {file}: {e}")
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 4: Data Cleaning Summary", ln=True)
    pdf.set_font("Arial", size=10)
    for file, count in cleaned_counts.items():
        pdf.cell(0, 10, f"Cleaned {file}: {count} laps", ln=True)
    pdf.ln(5)
    print()

    # Step 5: Driver Performance
    print("--- Step 5: Driver Performance ---")
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 5: Driver Performance", ln=True)
    df = driver_performance_df[['Driver', 'FastestLapTime', 'TheoreticalLapTime', 'BasePace', 'DegradationSlope', 'Team']]
    print("Top 5 by Fastest Lap Time:")
    display(df.sort_values('FastestLapTime').head(5))
    add_df_to_pdf(df.sort_values('FastestLapTime').head(5), "Top 5 by Fastest Lap Time")
    fig, ax = plt.subplots(figsize=(12, 6))
    bar_width = 0.35
    index = range(5)
    ax.bar([i - bar_width/2 for i in index], df['FastestLapTime'].head(5), bar_width, label='Fastest Lap Time', color='blue')
    ax.bar([i + bar_width/2 for i in index], df['TheoreticalLapTime'].head(5), bar_width, label='Theoretical Lap Time', color='orange')
    ax.set_xlabel('Driver')
    ax.set_ylabel('Lap Time (seconds)')
    ax.set_title(f'Fastest vs. Theoretical Lap Times ({year} Round {round_number})')
    ax.set_xticks(index)
    ax.set_xticklabels(df['Driver'].head(5), rotation=45, ha='right')
    ax.legend()
    ax.grid(True)
    add_plot_to_pdf(fig, "Fastest vs. Theoretical Lap Times")
    print()

    # Step 6: Telemetry Metrics
    print("--- Step 6: Telemetry Metrics ---")
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 6: Telemetry Metrics", ln=True)
    df = telemetry_metrics_df[['Driver', 'MaxSpeed', 'ThrottleTime', 'BrakingIntensity', 'Team']]
    print("Top 5 by Max Speed:")
    display(df.sort_values('MaxSpeed', ascending=False).head(5))
    add_df_to_pdf(df.sort_values('MaxSpeed', ascending=False).head(5), "Top 5 by Max Speed")
    fig, axs = plt.subplots(3, 1, figsize=(12, 8))
    metrics = ['MaxSpeed', 'ThrottleTime', 'BrakingIntensity']
    titles = ['Max Speed (km/h)', '% Time at Full Throttle', 'Avg Braking Intensity (0-1)']
    colors = ['blue', 'orange', 'green']
    for i, (metric, title, color) in enumerate(zip(metrics, titles, colors)):
        plot_data = df[df[metric] > 0] if metric == 'ThrottleTime' else df
        sorted_data = plot_data.sort_values(metric, ascending=False).head(5)
        axs[i].bar(sorted_data['Driver'], sorted_data[metric], color=color)
        axs[i].set_title(f'Telemetry Metrics - {title}')
        axs[i].set_xlabel('Driver')
        axs[i].set_ylabel(title)
        axs[i].tick_params(axis='x', rotation=45)
        axs[i].grid(True)
    plt.tight_layout()
    add_plot_to_pdf(fig, "Telemetry Metrics")
    print()

    # Step 7: Sandbagging Analysis
    print("--- Step 7: Sandbagging Analysis ---")
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 7: Sandbagging Analysis", ln=True)
    df = sandbag_analysis_df[['Driver', 'Team', 'FastestLapTime', 'MaxSpeed', 'PredictionDiscrepancy', 'SandbagFlag']]
    print("Potential Sandbaggers:")
    display(df[df['SandbagFlag']].sort_values('PredictionDiscrepancy', ascending=False))
    add_df_to_pdf(df[df['SandbagFlag']].sort_values('PredictionDiscrepancy', ascending=False), "Potential Sandbaggers")
    fig, ax = plt.subplots(figsize=(12, 6))
    colors = ['blue' if not flag else 'red' for flag in df['SandbagFlag']]
    ax.scatter(df['FastestLapTime'], df['MaxSpeed'], c=colors, alpha=0.6)
    ax.axhline(y=df['MaxSpeed'].mean(), color='gray', linestyle='--', label='Mean Max Speed')
    ax.set_xlabel('Fastest Lap Time (seconds)')
    ax.set_ylabel('Max Speed (km/h)')
    ax.set_title(f'Sandbagging Analysis ({year} Round {round_number})')
    for i, row in df.iterrows():
        ax.text(row['FastestLapTime'], row['MaxSpeed'], row['Driver'], fontsize=8, ha='right')
    ax.legend()
    ax.grid(True)
    add_plot_to_pdf(fig, "Sandbagging Analysis: Fastest Lap Time vs. Max Speed")
    print()

    # Step 8: Track Evolution and Weather
    print("--- Step 8: Track Evolution and Weather ---")
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 8: Track Evolution and Weather", ln=True)
    df = track_evolution_df[['Session', 'AvgLapTime', 'StdLapTime', 'LapCount']]
    print("Average Lap Times Across Sessions:")
    display(df)
    add_df_to_pdf(df, "Average Lap Times Across Sessions")
    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.plot(df['Session'], df['AvgLapTime'], marker='o', color='blue', label='Avg Lap Time (s)')
    ax1.fill_between(df['Session'], df['AvgLapTime'] - df['StdLapTime'] / 2, df['AvgLapTime'] + df['StdLapTime'] / 2, color='blue', alpha=0.2, label='Std Dev (±0.5σ)')
    ax1.set_xlabel('Session')
    ax1.set_ylabel('Average Lap Time (seconds)', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.set_xticks(df['Session'])
    ax1.set_xticklabels(['P1', 'P2', 'P3'], rotation=45)
    ax1.grid(True)
    ax1.legend(loc='upper left')
    add_plot_to_pdf(fig, "Track Evolution: Average Lap Times")
    print()

    # Step 9: Track Characteristics
    print("--- Step 9: Track Characteristics ---")
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 9: Track Characteristics", ln=True)
    track_char_df = pd.read_csv(os.path.join(BASE_DIR, data_dir, f"{year}_R{round_number}_track_characteristics.csv"))
    print("Track Characteristics:")
    display(track_char_df)
    add_df_to_pdf(track_char_df, "Track Characteristics")
    df = track_char_drivers_df[['Driver', 'Team', 'StraightSpeedPotential', 'CorneringPotential', 'AvgSpeed', 'OverallFit', 'FitRank']]
    print("Top 5 Drivers by Track Fit:")
    display(df.sort_values('FitRank').head(5))
    add_df_to_pdf(df.sort_values('FitRank').head(5), "Top 5 Drivers by Track Fit")
    fig, ax = plt.subplots(figsize=(12, 6))
    scatter = ax.scatter(df['StraightSpeedPotential'], df['AvgSpeed'], c=df['FitRank'], cmap='viridis')
    plt.colorbar(scatter, label='Fit Rank')
    for i, row in df.iterrows():
        ax.text(row['StraightSpeedPotential'], row['AvgSpeed'], row['Driver'], fontsize=8, ha='right')
    ax.set_xlabel('Max Straight Speed (km/h)')
    ax.set_ylabel('Average Speed (km/h)')
    ax.set_title(f'Aeromap ({year} Round {round_number})')
    ax.grid(True)
    add_plot_to_pdf(fig, "Aeromap: Max Speed vs. Average Speed")
    print()

    # Step 10: Predictions
    print("--- Step 10: Predictions ---")
    pdf.add_page()
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Step 10: Predictions", ln=True)
    quali_df = quali_prediction_df[['Driver', 'Team', 'AdjustedLapTime', 'QualiPosition']]
    print("Predicted Qualifying Top 5:")
    display(quali_df.sort_values('QualiPosition').head(5))
    add_df_to_pdf(quali_df.sort_values('QualiPosition').head(5), "Predicted Qualifying Top 5")
    race_df = race_prediction_df[['Driver', 'Team', 'GridPosition', 'AdjustedRaceTime', 'FinalPosition']]
    print("Predicted Race Top 10:")
    display(race_df.sort_values('FinalPosition').head(10))
    add_df_to_pdf(race_df.sort_values('FinalPosition').head(10), "Predicted Race Top 10")
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 10, f"Predicted Fastest Lap Driver: {fastest_lap_pred}", ln=True)
    print(f"Predicted Fastest Lap Driver: {fastest_lap_pred}\n")

    # Save PDF
    pdf_output = os.path.join(report_dir, f"{year}_R{round_number}_pre_qualifying_report.pdf")
    pdf.output(pdf_output)
    print(f"Report saved to {pdf_output}")

# Standalone execution for debugging (runs only if required variables are in scope)
if all(var in globals() for var in ['target_info', 'driver_performance_df', 'telemetry_metrics_df', 'sandbag_analysis_df', 'track_evolution_df', 'track_char_drivers_df', 'quali_prediction_df', 'race_prediction_df', 'fastest_lap_pred']):
    generate_comprehensive_report(
        target_info, driver_performance_df, telemetry_metrics_df, sandbag_analysis_df,
        track_evolution_df, track_char_drivers_df, quali_prediction_df, race_prediction_df, fastest_lap_pred
    )
else:
    print("Warning: Required variables (target_info, driver_performance_df, telemetry_metrics_df, sandbag_analysis_df, track_evolution_df, track_char_drivers_df, quali_prediction_df, race_prediction_df, fastest_lap_pred) not found. Run Cells 2, 5, 6, 7, 8, 9, and 10 first for standalone execution.")

In [ ]:
# Step 11: Final Summary Report
import pandas as pd
import os
import fastf1
import numpy as np

def generate_summary_report(config, quali_prediction_df, race_prediction_df, fastest_lap_pred):
    year = config.get('year')
    round_number = config.get('round')
    data_dir = os.path.join("raw_data", str(year), f"R{round_number}")
    event_name = config.get('event_name', "Unknown Event")
    BASE_DIR = os.getcwd()

    print("Debug: Verifying input DataFrames...")
    print(f"quali_prediction_df shape: {quali_prediction_df.shape if quali_prediction_df is not None else 'None'}")
    print(f"race_prediction_df shape: {race_prediction_df.shape if race_prediction_df is not None else 'None'}")

    round_data_dir = os.path.join(BASE_DIR, data_dir)
    driver_performance_file = os.path.join(round_data_dir, f"{year}_R{round_number}_driver_performance.csv")
    try:
        driver_performance = pd.read_csv(driver_performance_file)
    except Exception as e:
        print(f"Error loading driver performance data: {e}")
        driver_performance = pd.DataFrame()

    # Load pit_stop_loss
    pit_stop_file = os.path.join(BASE_DIR, "pit_stop_loss_by_track.csv")
    if os.path.exists(pit_stop_file):
        pit_stop_df = pd.read_csv(pit_stop_file)
        if event_name in pit_stop_df['Track'].values:
            pit_stop_loss = pit_stop_df.loc[pit_stop_df['Track'] == event_name, 'PitStopLoss'].values[0]
            print(f"Loaded pit_stop_loss for comparison: {pit_stop_loss:.2f}s for {event_name}")
        else:
            pit_stop_loss = 20.0
            print(f"No pit_stop_loss found for {event_name} in file, using default: {pit_stop_loss}s")
    else:
        pit_stop_loss = 20.0
        print(f"Pit stop loss file not found, using default: {pit_stop_loss}s")

    # Fetch actual results
    try:
        quali_session = fastf1.get_session(year, round_number, 'Q')
        quali_session.load()
        race_session = fastf1.get_session(year, round_number, 'R')
        race_session.load()
        quali_results = quali_session.results
        race_results = race_session.results
        actual_quali_top_5 = quali_results['Abbreviation'].head(5).tolist()
        actual_race_top_10 = race_results['Abbreviation'].head(10).tolist()
        laps = race_session.laps
        fastest_lap_driver = laps.pick_fastest()['Driver']

        actual_race_pace = laps[laps['PitInTime'].isna() & laps['LapTime'].notna()].groupby('Driver')['LapTime'].mean().dt.total_seconds().reset_index()
        actual_race_pace.columns = ['Driver', 'ActualRacePace']

        actual_deg = laps[laps['TyreLife'].notna() & laps['LapTime'].notna()].groupby('Driver').apply(
            lambda x: np.polyfit(x['TyreLife'], x['LapTime'].dt.total_seconds(), 1)[0] * 100 if len(x) > 5 else np.nan,
            include_groups=False
        ).reset_index()
        actual_deg.columns = ['Driver', 'ActualDegradationSlope']

        actual_overtakes = race_results[['Abbreviation', 'GridPosition', 'Position']].dropna()
        actual_overtakes['Position'] = actual_overtakes['Position'].astype(int)
        actual_overtakes['Overtakes'] = actual_overtakes['GridPosition'] - actual_overtakes['Position']
        actual_overtakes = actual_overtakes[['Abbreviation', 'Overtakes']]
        actual_overtakes.columns = ['Driver', 'ActualOvertakes']

        pit_in_laps = laps[laps['PitInTime'].notna()].copy()
        if not pit_in_laps.empty:
            avg_clean_lap = actual_race_pace.set_index('Driver')['ActualRacePace']
            pit_in_laps['PitDuration'] = pit_in_laps.apply(
                lambda row: row['LapTime'].total_seconds() - avg_clean_lap.get(row['Driver'], row['LapTime'].total_seconds()) if pd.notna(row['LapTime']) else np.nan,
                axis=1
            )
            pit_stops = pit_in_laps.groupby('Driver').agg(
                NumPitStops=pd.NamedAgg(column='PitInTime', aggfunc='count'),
                TotalPitTime=pd.NamedAgg(column='PitDuration', aggfunc='sum'),
                AvgPitTime=pd.NamedAgg(column='PitDuration', aggfunc='mean')
            ).reset_index()
            print("Debug: Sample pit stop durations:")
            print(pit_in_laps[['Driver', 'LapNumber', 'LapTime', 'PitDuration']].head())
        else:
            print("Warning: No pit-in laps found. Pit stop data unavailable.")
            pit_stops = pd.DataFrame(columns=['Driver', 'NumPitStops', 'TotalPitTime', 'AvgPitTime'])

        actual_tyres = laps.groupby('Driver')['Compound'].unique().reset_index()
        actual_tyres['ActualTyreStrategy'] = actual_tyres['Compound'].apply(lambda x: '-'.join(x))

        track_pit_stop_loss = pit_stops['AvgPitTime'].mean() if not pit_stops.empty else 20.0
        print(f"Calculated pit_stop_loss for {event_name}: {track_pit_stop_loss:.2f}s")
        if os.path.exists(pit_stop_file):
            pit_stop_df = pd.read_csv(pit_stop_file)
            if event_name in pit_stop_df['Track'].values:
                pit_stop_df.loc[pit_stop_df['Track'] == event_name, 'PitStopLoss'] = track_pit_stop_loss
            else:
                pit_stop_df = pd.concat([pit_stop_df, pd.DataFrame({'Track': [event_name], 'PitStopLoss': [track_pit_stop_loss]})], ignore_index=True)
        else:
            pit_stop_df = pd.DataFrame({'Track': [event_name], 'PitStopLoss': [track_pit_stop_loss]})
        pit_stop_df.to_csv(pit_stop_file, index=False)
        print(f"Saved pit_stop_loss to {pit_stop_file}")

        print(f"Dynamically fetched actual qualifying top 5: {actual_quali_top_5}")
        print(f"Dynamically fetched actual race top 10: {actual_race_top_10}")
        print(f"Dynamically fetched actual fastest lap driver: {fastest_lap_driver}")
    except Exception as e:
        print(f"Error fetching actual results from FastF1: {e}")
        print("Falling back to placeholder results.")
        actual_quali_top_5 = ['Unknown'] * 5
        actual_race_top_10 = ['Unknown'] * 10
        fastest_lap_driver = 'Unknown'
        actual_race_pace = pd.DataFrame()
        actual_deg = pd.DataFrame()
        actual_overtakes = pd.DataFrame()
        pit_stops = pd.DataFrame()
        actual_tyres = pd.DataFrame()

    print("\n=== Comprehensive Summary Report ===")
    print(f"\nEvent: {year} Round {round_number} - {event_name}")

    print("\n--- Qualifying Analysis ---")
    print("Actual Qualifying Top 5:", actual_quali_top_5)
    print("Predicted Qualifying Top 5:")
    quali_pred_df = quali_prediction_df[['Driver', 'AdjustedLapTime', 'QualiPosition']].sort_values('QualiPosition').head(5)
    display(quali_pred_df)

    print("\n--- Race Analysis ---")
    print("Actual Race Top 10:", actual_race_top_10)
    print("Predicted Race Top 10:")
    race_pred_df = race_prediction_df[['Driver', 'Team', 'GridPosition', 'AdjustedRaceTime', 'FinalPosition']].sort_values('FinalPosition').head(10)
    display(race_pred_df)

    print("\n--- Detailed Prediction vs. Actual Comparison ---")
    comparison_df = race_prediction_df[['Driver', 'GridPosition', 'FinalPosition', 'BasePace', 'DegradationSlope', 'FuelPenalty', 'TrafficPenalty', 'CompoundStrategy', 'StintLengths']].merge(
        actual_race_pace, on='Driver', how='left'
    ).merge(
        actual_deg, on='Driver', how='left'
    ).merge(
        actual_overtakes, on='Driver', how='left'
    ).merge(
        pit_stops, on='Driver', how='left'
    ).merge(
        actual_tyres[['Driver', 'ActualTyreStrategy']], on='Driver', how='left'
    )
    comparison_df['PaceError'] = comparison_df['BasePace'] - comparison_df['ActualRacePace']
    comparison_df['DegError'] = comparison_df['DegradationSlope'] - comparison_df['ActualDegradationSlope']
    comparison_df['PositionError'] = comparison_df['FinalPosition'] - comparison_df['ActualOvertakes'].fillna(0).apply(lambda x: len(actual_race_top_10) + 1 - x if x > 0 else len(actual_race_top_10) + 1)
    comparison_df['PitStopCountError'] = comparison_df['NumPitStops'].fillna(0) - comparison_df['CompoundStrategy'].apply(lambda x: len(x.split('-')) - 1)
    comparison_df['PitTimeError'] = (comparison_df['NumPitStops'].fillna(0) * pit_stop_loss) - comparison_df['TotalPitTime'].fillna(0)
    display(comparison_df)

    results_file = os.path.join(round_data_dir, f"{year}_R{round_number}_summary_results.csv")
    comparison_df.to_csv(results_file, index=False)
    print(f"\nSaved detailed results to {results_file}")

    print("\n--- Insights for Improvement ---")
    print("1. **Race Pace Errors**:")
    pace_error_mean = comparison_df['PaceError'].mean()
    print(f"Mean Pace Error: {pace_error_mean:.2f} seconds")
    if abs(pace_error_mean) > 2:
        print(f" - Suggestion: Adjust BasePace calculation. Current model {'overestimates' if pace_error_mean > 0 else 'underestimates'} race pace by {abs(pace_error_mean):.2f}s.")

    print("2. **Degradation Errors**:")
    deg_error_mean = comparison_df['DegError'].mean()
    print(f"Mean Degradation Error: {deg_error_mean:.2f}%")
    if abs(deg_error_mean) > 5:
        print(f" - Suggestion: Refine DegradationSlope estimation. Model {'overestimates' if deg_error_mean > 0 else 'underestimates'} tyre wear by {abs(deg_error_mean):.2f}%.")

    print("3. **Overtaking Accuracy**:")
    overtake_error_mean = comparison_df['ActualOvertakes'].fillna(0) - (comparison_df['GridPosition'] - comparison_df['FinalPosition'])
    print(f"Mean Overtaking Error: {overtake_error_mean.mean():.2f} positions")
    if abs(overtake_error_mean.mean()) > 2:
        print(f" - Suggestion: Adjust overtaking_penalty (0.5s) or TrafficPenalty. Predictions {'underestimate' if overtake_error_mean.mean() > 0 else 'overestimate'} overtaking by {abs(overtake_error_mean.mean()):.2f} positions.")

    print("4. **Pit Stop Strategy**:")
    pit_count_error_mean = comparison_df['PitStopCountError'].mean()
    print(f"Mean Pit Stop Count Error: {pit_count_error_mean:.2f} stops")
    if abs(pit_count_error_mean) > 0.5:
        print(f" - Suggestion: Refine stint length logic. Model {'overestimates' if pit_count_error_mean < 0 else 'underestimates'} pit stops by {abs(pit_count_error_mean):.2f}.")

    print("5. **Pit Stop Time**:")
    pit_time_error_mean = comparison_df['PitTimeError'].mean()
    print(f"Mean Pit Time Error: {pit_time_error_mean:.2f} seconds")
    if abs(pit_time_error_mean) > 5:
        print(f" - Suggestion: Adjust pit_stop_loss ({pit_stop_loss:.2f}s). Actual pit times are {'shorter' if pit_time_error_mean > 0 else 'longer'} by {abs(pit_time_error_mean):.2f}s.")

    print("\n--- Driver Performance Metrics ---")
    print("Fastest Lap Times:")
    display(driver_performance[['Driver', 'FastestLapTime', 'Team']].sort_values('FastestLapTime'))

    print("\n--- F1 Fantasy League Scoring ---")
    quali_top_5_pred = quali_prediction_df.sort_values('QualiPosition').head(5)['Driver'].tolist()
    race_top_10_pred = race_prediction_df.sort_values('FinalPosition').head(10)['Driver'].tolist()
    podium_pred = race_top_10_pred[:3]
    podium_actual = actual_race_top_10[:3]

    podium_points = 0
    podium_details = []
    for position, (pred, act) in enumerate(zip(podium_pred, podium_actual), 1):
        points = 0
        if pred in actual_race_top_10:
            points += 1
            podium_details.append(f"P{position}           {act:<15} {pred:<15} 1pt (in top 10)")
        if pred == act:
            points += 1
            podium_details.append(f"P{position}           {act:<15} {pred:<15} 1pt (exact position)")
        if points == 0:
            podium_details.append(f"P{position}           {act:<15} {pred:<15} 0pt")
        podium_points += points
    if set(podium_pred).issubset(set(podium_actual)):
        podium_points += 2
        podium_details.append(f"Podium Bonus  {str(podium_actual):<15} {str(podium_pred):<15} 2pt (all 3 correct)")
    else:
        podium_details.append(f"Podium Bonus  {str(podium_actual):<15} {str(podium_pred):<15} 0pt (not all 3 correct)")

    top_10_points = 0
    top_10_details = []
    for position, (pred, act) in enumerate(zip(race_top_10_pred[3:], actual_race_top_10[3:]), 4):
        points = 0
        if pred in actual_race_top_10:
            points += 1
            top_10_details.append(f"P{position}           {act:<15} {pred:<15} 1pt (in top 10)")
            if pred == act:
                points += 0.5
                top_10_details.append(f"P{position}           {act:<15} {pred:<15} 0.5pt (exact position)")
        else:
            top_10_details.append(f"P{position}           {act:<15} {pred:<15} 0pt")
        top_10_points += points

    fastest_lap_points = 1 if fastest_lap_pred == fastest_lap_driver else 0
    fastest_lap_detail = f"Fastest Lap   {fastest_lap_driver:<15} {fastest_lap_pred:<15} {fastest_lap_points}pt"

    total_points = podium_points + top_10_points + fastest_lap_points

    print("\nRace Results")
    print(f"{'Actual':<15} {'You':<15} {'Points':<15}")
    print("-" * 45)
    pole_pred = quali_prediction_df.sort_values('QualiPosition').head(1)['Driver'].tolist()[0]
    print(f"Pole          {actual_quali_top_5[0]:<15} {pole_pred:<15} 2pt" if pole_pred == actual_quali_top_5[0] else f"Pole          {actual_quali_top_5[0]:<15} {pole_pred:<15} 0pt")
    print(fastest_lap_detail)
    for detail in podium_details:
        print(detail)
    for detail in top_10_details:
        print(detail)

    print(f"\nTotal Fantasy League Points: {total_points}")
    print(f"Maximum Possible Points: 20")

# Standalone execution for debugging (runs only if required variables are in scope)
if all(var in globals() for var in ['target_info', 'quali_prediction_df', 'race_prediction_df', 'fastest_lap_pred']):
    generate_summary_report(target_info, quali_prediction_df, race_prediction_df, fastest_lap_pred)
else:
    print("Warning: Required variables (target_info, quali_prediction_df, race_prediction_df, fastest_lap_pred) not found. Run Cells 2 and 10 first for standalone execution.")

In [ ]:
# Step 12: Batch Process for Rounds or Next Race
import os
import fastf1
import pandas as pd
import logging
from datetime import datetime

# Configure logging
BASE_DIR = os.getcwd()
log_dir = os.path.join(BASE_DIR, "logs")
os.makedirs(log_dir, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(log_dir, f"batch_process_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# List of required functions from Cells 2-11
required_functions = [
    'get_target_event', 'download_practice_data', 'clean_and_aggregate_data',
    'compute_driver_performance', 'analyze_telemetry_metrics', 'investigate_sandbagging',
    'evaluate_track_evolution', 'analyze_track_characteristics', 'predict_race_outcomes',
    'generate_comprehensive_report', 'generate_summary_report'
]

def batch_process_rounds(year=2024, rounds=None):
    # Check if all required functions are defined
    missing_functions = [func for func in required_functions if func not in globals()]
    if missing_functions:
        error_msg = f"Error: The following functions are not defined: {missing_functions}. Please run Cells 2-11 first."
        logger.error(error_msg)
        print(error_msg)
        return {}

    fastf1.Cache.enable_cache(os.path.join(BASE_DIR, "fastf1_cache"))
    fastf1.set_log_level('ERROR')
    if rounds is None:
        mode = "next race"
        logger.info(f"Starting batch process for {year}, Next Race mode")
        print(f"\n=== Starting Batch Process for {year}, Next Race ===")
    else:
        mode = "specified rounds"
        logger.info(f"Starting batch process for {year}, Rounds {list(rounds)}")
        print(f"\n=== Starting Batch Process for {year}, Rounds {list(rounds)} ===")

    try:
        schedule = fastf1.get_event_schedule(year)
        logger.info(f"Fetched {year} schedule with {len(schedule)} events")
        print(f"Schedule fetched: {len(schedule)} events")
        display(schedule[['RoundNumber', 'EventName', 'EventDate']])
    except Exception as e:
        logger.error(f"Failed to fetch event schedule: {e}")
        print(f"Error fetching event schedule: {e}")
        return {}

    results = {}
    if rounds is None:
        target_info = get_target_event(manual_year=year, manual_round=None, skip_timing_validation=True)
        round_number = target_info['round']
        rounds_to_process = [round_number]
        logger.info(f"Next race identified: Round {round_number}")
        print(f"Next race identified: Round {round_number}")
    else:
        rounds_to_process = rounds

    for round_number in rounds_to_process:
        logger.info(f"Processing Round {round_number}")
        print(f"\n=== Processing {year} Round {round_number} ===")

        try:
            # Step 2
            print("Step 2: Fetching target event info...")
            if rounds is not None:
                target_info = get_target_event(manual_year=year, manual_round=round_number, skip_timing_validation=True)
            event_name = target_info['event_name']
            logger.info(f"Target event: {event_name} (Round {round_number})")
            print(f"Event: {event_name}")
            print(f"Target Info: {target_info}")

            # Step 3
            print("Step 3: Downloading practice data...")
            session_data = download_practice_data(target_info)
            if not session_data:
                logger.error(f"No session data for Round {round_number}")
                print("Error: No session data retrieved")
                continue
            logger.info(f"Session data keys: {list(session_data.keys())}")
            print(f"Session data retrieved for: {list(session_data.keys())}")
            for session, data in session_data.items():
                print(f"{session} - Laps: {len(data['laps'])}, Weather: {len(data['weather'])}, Indicators: {len(data['indicators'])}")

            # Step 4
            print("Step 4: Cleaning and aggregating data...")
            cleaned_data, driver_summary = clean_and_aggregate_data(target_info, session_data)
            if cleaned_data == {}:
                logger.error(f"Cleaning failed for Round {round_number}")
                print("Error: Cleaning failed, no cleaned data")
                continue
            logger.info(f"Cleaned data keys: {list(cleaned_data.keys())}")
            print(f"Cleaned data keys: {list(cleaned_data.keys())}")
            print("Driver Summary (Top 5):")
            display(driver_summary.head())

            # Step 5
            print("Step 5: Computing driver performance...")
            driver_performance_df = compute_driver_performance(cleaned_data, year, round_number)
            if driver_performance_df.empty:
                logger.error(f"Driver performance analysis failed for Round {round_number}")
                print("Error: Driver performance DataFrame is empty")
                continue
            logger.info(f"Driver performance rows: {len(driver_performance_df)}")
            print(f"Driver performance rows: {len(driver_performance_df)}")
            print("Driver Performance (Top 5 by FastestLapTime):")
            display(driver_performance_df.sort_values('FastestLapTime').head())

            # Step 6
            print("Step 6: Analyzing telemetry metrics...")
            telemetry_metrics_df = analyze_telemetry_metrics(target_info)
            if telemetry_metrics_df.empty:
                logger.warning(f"Telemetry metrics empty for Round {round_number}, proceeding")
                print("Warning: Telemetry metrics DataFrame is empty, proceeding")
            logger.info(f"Telemetry metrics rows: {len(telemetry_metrics_df)}")
            print(f"Telemetry metrics rows: {len(telemetry_metrics_df)}")
            if not telemetry_metrics_df.empty:
                print("Telemetry Metrics (Top 5 by MaxSpeed):")
                display(telemetry_metrics_df.sort_values('MaxSpeed', ascending=False).head())

            # Step 7
            print("Step 7: Investigating sandbagging...")
            sandbag_analysis_df = investigate_sandbagging(target_info)
            if sandbag_analysis_df.empty:
                logger.warning(f"Sandbagging analysis empty for Round {round_number}, proceeding")
                print("Warning: Sandbagging analysis DataFrame is empty, proceeding")
            logger.info(f"Sandbag analysis rows: {len(sandbag_analysis_df)}")
            print(f"Sandbag analysis rows: {len(sandbag_analysis_df)}")
            if not sandbag_analysis_df.empty:
                print("Potential Sandbaggers:")
                display(sandbag_analysis_df[sandbag_analysis_df['SandbagFlag']].sort_values('PredictionDiscrepancy', ascending=False))

            # Step 8
            print("Step 8: Evaluating track evolution...")
            track_evolution_df = evaluate_track_evolution(target_info, cleaned_data)
            if track_evolution_df.empty:
                logger.warning(f"Track evolution empty for Round {round_number}, proceeding")
                print("Warning: Track evolution DataFrame is empty, proceeding")
            logger.info(f"Track evolution rows: {len(track_evolution_df)}")
            print(f"Track evolution rows: {len(track_evolution_df)}")
            if not track_evolution_df.empty:
                print("Track Evolution:")
                display(track_evolution_df)

            # Step 9
            print("Step 9: Analyzing track characteristics...")
            track_char_drivers_df = analyze_track_characteristics(target_info, cleaned_data)
            if track_char_drivers_df.empty:
                logger.warning(f"Track characteristics empty for Round {round_number}, proceeding")
                print("Warning: Track characteristics DataFrame is empty, proceeding")
            logger.info(f"Track characteristics drivers rows: {len(track_char_drivers_df)}")
            print(f"Track characteristics drivers rows: {len(track_char_drivers_df)}")
            if not track_char_drivers_df.empty:
                print("Track Characteristics Drivers (Top 5 by FitRank):")
                display(track_char_drivers_df.sort_values('FitRank').head())

            # Step 10
            print("Step 10: Predicting race outcomes...")
            quali_prediction_df, race_prediction_df, fastest_lap_pred = predict_race_outcomes(
                target_info, driver_performance_df, telemetry_metrics_df, sandbag_analysis_df,
                track_evolution_df, track_char_drivers_df
            )
            if quali_prediction_df.empty or race_prediction_df.empty:
                logger.error(f"Prediction failed for Round {round_number}")
                print("Error: Prediction DataFrames are empty")
                continue
            logger.info(f"Quali prediction rows: {len(quali_prediction_df)}, Race prediction rows: {len(race_prediction_df)}")
            print(f"Quali prediction rows: {len(quali_prediction_df)}, Race prediction rows: {len(race_prediction_df)}")
            print("Qualifying Prediction (Top 5):")
            display(quali_prediction_df.sort_values('QualiPosition').head())
            print("Race Prediction (Top 10):")
            display(race_prediction_df.sort_values('FinalPosition').head(10))
            print(f"Fastest Lap Prediction: {fastest_lap_pred}")

            # Step 10.5
            print("Step 10.5: Generating pre-qualifying report...")
            generate_comprehensive_report(
                target_info, driver_performance_df, telemetry_metrics_df, sandbag_analysis_df,
                track_evolution_df, track_char_drivers_df, quali_prediction_df, race_prediction_df, fastest_lap_pred
            )
            logger.info(f"Pre-qualifying report generated for Round {round_number}")
            print(f"Pre-qualifying report generated: {os.path.join(BASE_DIR, 'reports', str(year), f'R{round_number}', f'{year}_R{round_number}_pre_qualifying_report.pdf')}")

            # Step 11
            print("Step 11: Generating summary report with actual results...")
            generate_summary_report(target_info, quali_prediction_df, race_prediction_df, fastest_lap_pred)
            logger.info(f"Summary report generated for Round {round_number}")
            print(f"Summary report saved: {os.path.join(BASE_DIR, 'raw_data', str(year), f'R{round_number}', f'{year}_R{round_number}_summary_results.csv')}")

            results[round_number] = {
                'target_info': target_info,
                'session_data': session_data,
                'cleaned_data': cleaned_data,
                'driver_summary': driver_summary,
                'driver_performance_df': driver_performance_df,
                'telemetry_metrics_df': telemetry_metrics_df,
                'sandbag_analysis_df': sandbag_analysis_df,
                'track_evolution_df': track_evolution_df,
                'track_char_drivers_df': track_char_drivers_df,
                'quali_prediction_df': quali_prediction_df,
                'race_prediction_df': race_prediction_df,
                'fastest_lap_pred': fastest_lap_pred
            }
            print(f"Results stored for Round {round_number}: {list(results[round_number].keys())}")

        except Exception as e:
            logger.error(f"Error processing Round {round_number}: {e}")
            print(f"Error processing Round {round_number}: {e}")
            continue

    logger.info("Batch processing completed")
    print("\n=== Batch Processing Completed ===")
    print(f"Processed rounds: {list(results.keys())}")
    return results

# Test with Round 4
print("Starting batch process for Round 4...")
batch_results = batch_process_rounds(year=2024, rounds=range(5,6))

# Test with next race (commented out, uncomment to try after fixing)
# print("Starting batch process for next race...")
# batch_results = batch_process_rounds(year=2025)

In [ ]:
# Step 13: Consolidate Data into SQLite Database
import os
import pandas as pd
import sqlite3
from datetime import datetime

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "raw_data")
DB_PATH = os.path.join(BASE_DIR, "f1_fantasy.db")

def consolidate_data(rounds=None, force_reimport=False):
    print(f"\n=== Consolidating Data into {DB_PATH} ===")
    if rounds is None:
        print("Auto-detecting all rounds from raw_data directory...")
    else:
        print(f"Processing specified rounds: {rounds}")

    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Table schemas (unchanged except for clarity)
    tables = {
        'events': """
            CREATE TABLE events (
                RoundNumber INTEGER,
                Year INTEGER,
                EventName TEXT,
                EventDate TEXT,
                Location TEXT,
                TrackLength REAL,
                AvgSpeed REAL,
                DownforceLevel TEXT,
                PRIMARY KEY (Year, RoundNumber)
            )
        """,
        'driver_performance': """
            CREATE TABLE driver_performance (
                RoundNumber INTEGER,
                Year INTEGER,
                Driver TEXT,
                FastestLapTime REAL,
                TheoreticalLapTime REAL,
                Sector1Time REAL,
                Sector2Time REAL,
                Sector3Time REAL,
                BasePace REAL,
                LapTimeVar REAL,
                DegradationSlope REAL,
                LapTimeGap REAL,
                Team TEXT,
                PRIMARY KEY (Year, RoundNumber, Driver)
            )
        """,
        'telemetry_metrics': """
            CREATE TABLE telemetry_metrics (
                RoundNumber INTEGER,
                Year INTEGER,
                Driver TEXT,
                MaxSpeed REAL,
                ThrottleTime REAL,
                BrakingIntensity REAL,
                Team TEXT,
                PRIMARY KEY (Year, RoundNumber, Driver)
            )
        """,
        'sandbag_analysis': """
            CREATE TABLE sandbag_analysis (
                RoundNumber INTEGER,
                Year INTEGER,
                Driver TEXT,
                FastestLapTime REAL,
                MaxSpeed REAL,
                ExpectedLapTime REAL,
                LapTimeDiscrepancy REAL,
                PredictedLapTime REAL,
                PredictionDiscrepancy REAL,
                SpeedRank REAL,
                LapTimeRank REAL,
                SandbagFlag INTEGER,
                ThrottleTime REAL,
                Team TEXT,
                PRIMARY KEY (Year, RoundNumber, Driver)
            )
        """,
        'track_evolution': """
            CREATE TABLE track_evolution (
                RoundNumber INTEGER,
                Year INTEGER,
                Session TEXT,
                AvgLapTime REAL,
                StdLapTime REAL,
                LapCount INTEGER,
                PRIMARY KEY (Year, RoundNumber, Session)
            )
        """,
        'track_characteristics': """
            CREATE TABLE track_characteristics (
                RoundNumber INTEGER,
                Year INTEGER,
                Driver TEXT,
                SpeedST REAL,
                Sector1Time REAL,
                Sector2Time REAL,
                Sector3Time REAL,
                LapTime REAL,
                StraightSpeedPotential REAL,
                CorneringPotential REAL,
                AvgSpeed REAL,
                OverallFit REAL,
                FitRank REAL,
                StraightZScore REAL,
                CorneringZScore REAL,
                Team TEXT,
                PRIMARY KEY (Year, RoundNumber, Driver)
            )
        """,
        'predictions': """
            CREATE TABLE predictions (
                RoundNumber INTEGER,
                Year INTEGER,
                Driver TEXT,
                QualiAdjustedLapTime REAL,
                QualiPosition INTEGER,
                RaceAdjustedRaceTime REAL,
                RaceFinalPosition INTEGER,
                FastestLapPred TEXT,
                Team TEXT,
                PRIMARY KEY (Year, RoundNumber, Driver)
            )
        """,
        'results': """
            CREATE TABLE results (
                RoundNumber INTEGER,
                Year INTEGER,
                Driver TEXT,
                GridPosition INTEGER,
                FinalPosition INTEGER,
                BasePace REAL,
                DegradationSlope REAL,
                ActualRacePace REAL,
                ActualDegradationSlope REAL,
                ActualOvertakes REAL,
                NumPitStops REAL,
                TotalPitTime REAL,
                AvgPitTime REAL,
                PaceError REAL,
                DegError REAL,
                PositionError REAL,
                PitStopCountError REAL,
                PitTimeError REAL,
                FuelPenalty REAL,
                TrafficPenalty REAL,
                CompoundStrategy TEXT,
                StintLengths TEXT,
                ActualTyreStrategy TEXT,
                TotalPoints REAL,
                Team TEXT,
                PRIMARY KEY (Year, RoundNumber, Driver)
            )
        """
    }

    # Drop tables if force_reimport is True
    if force_reimport:
        for table_name in tables.keys():
            cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
            print(f"Dropped table: {table_name}")

    # Create tables
    for table_name, create_stmt in tables.items():
        cursor.execute(create_stmt)
        print(f"Created table: {table_name}")

    # Get existing rounds (empty after drop, kept for non-force runs)
    existing_rounds = pd.read_sql_query("SELECT Year, RoundNumber FROM events", conn)
    existing_rounds_set = set(zip(existing_rounds['Year'], existing_rounds['RoundNumber']))

    # Auto-detect rounds if not specified
    if rounds is None:
        years = [y for y in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, y)) and y.isdigit()]
        print(f"Detected years: {years}")
        rounds_to_process = []
        for year in years:
            year_dir = os.path.join(DATA_DIR, year)
            round_dirs = [d for d in os.listdir(year_dir) if d.startswith('R') and os.path.isdir(os.path.join(year_dir, d))]
            for round_dir in round_dirs:
                round_number = int(round_dir.replace('R', ''))
                rounds_to_process.append((int(year), round_number))
    else:
        rounds_to_process = [(2024, r) for r in rounds]

    for year, round_number in rounds_to_process:
        if (year, round_number) in existing_rounds_set and not force_reimport:
            print(f"Skipping Round {round_number} of {year} - already in database")
            continue

        round_dir = os.path.join(DATA_DIR, str(year), f"R{round_number}")
        if not os.path.exists(round_dir):
            print(f"Warning: Directory {round_dir} not found, skipping")
            continue

        print(f"\nProcessing Round {round_number} of {year}...")

        # Events
        event_data = {
            'RoundNumber': round_number,
            'Year': year,
            'EventName': f"Round {round_number}",
            'EventDate': None,
            'Location': None,
            'TrackLength': None,
            'AvgSpeed': None,
            'DownforceLevel': None
        }
        try:
            track_char_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_track_characteristics.csv"))
            event_subset = track_char_df.iloc[0][['Track', 'LapLength', 'DownforceRequirement']].to_dict()
            event_data.update({
                'EventName': event_subset['Track'],
                'TrackLength': event_subset['LapLength'],
                'DownforceLevel': event_subset['DownforceRequirement']
            })
        except Exception as e:
            print(f"Warning: Could not enrich event data for Round {round_number} of {year}: {e}")
        pd.DataFrame([event_data]).to_sql('events', conn, if_exists='append', index=False)
        print(f"Loaded events: 1 row")

        # Driver Performance
        try:
            driver_performance_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_driver_performance.csv"))
            driver_performance_df['RoundNumber'] = round_number
            driver_performance_df['Year'] = year
            driver_performance_df.to_sql('driver_performance', conn, if_exists='append', index=False)
            print(f"Loaded driver_performance: {len(driver_performance_df)} rows")
        except Exception as e:
            print(f"Error loading driver_performance for Round {round_number} of {year}: {e}")

        # Telemetry Metrics
        try:
            telemetry_metrics_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_telemetry_metrics.csv"))
            telemetry_metrics_df['RoundNumber'] = round_number
            telemetry_metrics_df['Year'] = year
            telemetry_metrics_df.to_sql('telemetry_metrics', conn, if_exists='append', index=False)
            print(f"Loaded telemetry_metrics: {len(telemetry_metrics_df)} rows")
        except Exception as e:
            print(f"Error loading telemetry_metrics for Round {round_number} of {year}: {e}")

        # Sandbag Analysis
        try:
            sandbag_analysis_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_sandbag_analysis.csv"))
            sandbag_analysis_df['RoundNumber'] = round_number
            sandbag_analysis_df['Year'] = year
            sandbag_analysis_df.to_sql('sandbag_analysis', conn, if_exists='append', index=False)
            print(f"Loaded sandbag_analysis: {len(sandbag_analysis_df)} rows")
        except Exception as e:
            print(f"Error loading sandbag_analysis for Round {round_number} of {year}: {e}")

        # Track Evolution
        try:
            track_evolution_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_track_evolution.csv"))
            track_evolution_df['RoundNumber'] = round_number
            track_evolution_df['Year'] = year
            track_evolution_df.to_sql('track_evolution', conn, if_exists='append', index=False)
            print(f"Loaded track_evolution: {len(track_evolution_df)} rows")
        except Exception as e:
            print(f"Error loading track_evolution for Round {round_number} of {year}: {e}")

        # Track Characteristics
        try:
            track_char_drivers_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_track_characteristics_drivers.csv"))
            track_char_drivers_df['RoundNumber'] = round_number
            track_char_drivers_df['Year'] = year
            track_char_drivers_df.to_sql('track_characteristics', conn, if_exists='append', index=False)
            print(f"Loaded track_characteristics: {len(track_char_drivers_df)} rows")
        except Exception as e:
            print(f"Error loading track_characteristics for Round {round_number} of {year}: {e}")

        # Predictions
        try:
            quali_pred_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_quali_prediction.csv"))
            race_pred_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_race_prediction.csv"))
            print(f"Quali columns: {quali_pred_df.columns.tolist()}")
            print(f"Race columns: {race_pred_df.columns.tolist()}")
            predictions_df = quali_pred_df[['Driver', 'AdjustedLapTime', 'QualiPosition', 'Team']].merge(
                race_pred_df[['Driver', 'AdjustedRaceTime', 'FinalPosition']], on='Driver', how='outer'
            )
            predictions_df['RoundNumber'] = round_number
            predictions_df['Year'] = year
            predictions_df['FastestLapPred'] = None  # Placeholder
            predictions_df.rename(columns={
                'AdjustedLapTime': 'QualiAdjustedLapTime',
                'AdjustedRaceTime': 'RaceAdjustedRaceTime',  # Fixed renaming
                'FinalPosition': 'RaceFinalPosition'
            }, inplace=True)
            print(f"Predictions columns after merge: {predictions_df.columns.tolist()}")
            predictions_df.to_sql('predictions', conn, if_exists='append', index=False)
            print(f"Loaded predictions: {len(predictions_df)} rows")
        except Exception as e:
            print(f"Error loading predictions for Round {round_number} of {year}: {e}")

        # Results
        try:
            results_df = pd.read_csv(os.path.join(round_dir, f"{year}_R{round_number}_summary_results.csv"))
            results_df['RoundNumber'] = round_number
            results_df['Year'] = year
            results_df['TotalPoints'] = None  # Placeholder
            results_df.to_sql('results', conn, if_exists='append', index=False)
            print(f"Loaded results: {len(results_df)} rows")
        except Exception as e:
            print(f"Error loading results for Round {round_number} of {year}: {e}")

    # Verify table contents
    for table in tables.keys():
        count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0]['count']
        print(f"Total rows in {table}: {count}")

    conn.commit()
    conn.close()
    print(f"\nData consolidation completed. Database saved at {DB_PATH}")

# Run with force reimport
consolidate_data(force_reimport=True)